<a href="https://colab.research.google.com/github/danialk20/Analitica/blob/main/implementacion1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

Este notebook implementa un modelo de optimización para la gestión integral de transporte y distribución de materiales en una red logística. El modelo fue desarrollado como parte de un proyecto de consultoría y está diseñado para minimizar los costos operativos mientras se satisface la demanda de múltiples productos a través de una red de nodos, considerando restricciones de capacidad, inventario y características específicas de vehículos. Siendo este la primera de tres versiones de modelación del problema y escenarios.

### Objetivo del modelo

El modelo busca optimizar:
- **Flujos de materiales** entre nodos de la red
- **Asignación de vehículos** (propios y tercerizados)
- **Niveles de inventario** en cada ubicación
- **Gestión de unidades de transporte** (cabezotes y remolques)
- **Reposicionamiento de flota** para maximizar eficiencia

### Estructura del notebook

El notebook está organizado en las siguientes secciones principales:

1. **Librerías**: Instalación y carga de dependencias necesarias (PuLP, Gurobi, Pandas, etc.)

2. **Funciones**:
   - Lectura y validación de datos desde archivos .xlsx
   - Verificación de factibilidad del modelo
   - Cálculo de indicadores de desempeño (KPIs)
   - Exportación de resultados

3. **Cargue de Datos**: Importación y procesamiento de datos desde repositorio GitHub

4. **Implementación**:
   - Definición de conjuntos y parámetros
   - Declaración de variables de decisión
   - Formulación de restricciones
   - Definición de función objetivo
   - Resolución del modelo

5. **Análisis de Resultados**:
   - Indicadores clave de desempeño
   - Exportación a .xlsx
   - Diagnóstico de factibilidad

### Cómo Navegar este Notebook

- **Ejecución secuencial**: Ejecute las celdas en orden desde arriba hacia abajo
- **Primera ejecución**: Las librerías PuLP y Gurobi se instalarán automáticamente
- **Autenticación**: Se requiere token de GitHub para acceder a los datos (ya configurado)
- **Tiempo de ejecución**: La optimización puede tardar varios minutos dependiendo del tamaño del problema
- **Resultados**: Los outputs se generan en archivos .xlsx  descargables

### Requisitos del solver

El modelo utiliza **Gurobi** como solver principal a través de una licencia WLS (Web License Service). La configuración está incluida en el código. Alternativamente, se puede usar el solver CBC de código abierto (comentado en el código).

### Datos de entrada

El modelo requiere un archivo Excel estructurado con las siguientes hojas:
- Conjuntos, Nodos, Materiales, Vehículos
- Relaciones: MaterialesNodos, VehiculosNodos, Arcos
- Parámetros temporales: Demanda, MaterialesTransito, VehiculosTransito
- Capacidades: CapacidadPdn

### Outputs generados

- **Archivo .xlsx** con resultados detallados por variable
- **KPIs** de utilización, costos y eficiencia operativa
- **Diagnósticos** de factibilidad y restricciones activas



**Nota**: Para ejecutar este notebook, asegúrese de tener acceso a Google Colab y de que las credenciales de GitHub estén actualizadas.


---

# Librerías

A continuación se importan y definen las librerías que se usarán a lo largo de este notebook. Algunas librerías ya vienen preinstaladas en el entorno de Colab, mientras que otras (como PuLP) deben ser instaladas (al inicio de la sesión) usando pip.

In [ ]:
import importlib

def is_installed(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

if not is_installed('pulp'):
    !pip install pulp

if not is_installed('gurobipy'):
    !pip install gurobipy

# import gurobipy as gp
import pulp as pl
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import requests
import io
import math
import numpy as np
from collections import defaultdict
import os
from google.colab import files

---
# Funciones
En esta sección se definirán las funciones que han de usarse en las subsiguientes secciones.

## Funciones para la lectura de datos

Función para comprobar que el archivo de excel cargado tenga todas las hojas requeridas

In [ ]:
def verificar_hojas_excel(archivo_excel):
    """
    Verifica que el archivo Excel contenga todas las hojas requeridas por el modelo.

    Compara las hojas presentes en el archivo con una lista predefinida de hojas
    necesarias y genera un reporte visual con el porcentaje de completitud.

    Argumentos:
        archivo_excel (pd.ExcelFile): Objeto ExcelFile de pandas con el archivo cargado

    Imprime:
        - Barra de progreso visual con porcentaje de hojas encontradas
        - Lista de hojas faltantes (si las hay)
        - Mensaje de éxito si todas las hojas están presentes

    """

    hojas_requeridas = [
        'Conjuntos',
        'Nodos',
        'Materiales',
        'MaterialesNodos',
        'MaterialesTransito',
        'Vehiculos',
        'VehiculosNodos',
        'VehiculosTransito',
        'Arcos',
        'Demanda',
        'CapacidadPdn'
    ]
    hojas_presentes = archivo_excel.sheet_names
    hojas_encontradas = [hoja for hoja in hojas_requeridas if hoja in hojas_presentes]
    hojas_faltantes = [hoja for hoja in hojas_requeridas if hoja not in hojas_presentes]

    num_requeridas = len(hojas_requeridas)
    num_encontradas = len(hojas_encontradas)

    percentage = math.floor((num_encontradas / num_requeridas) * 100)

    # Representación visual del porcentaje
    filled_symbols = '■' * (percentage // 10)
    empty_symbols = '□' * (10 - (percentage // 10))

    print(f"Verificación de hojas:")
    print(f"{percentage}% de hojas requeridas encontradas: {filled_symbols}{empty_symbols}")

    if num_encontradas == num_requeridas:
        print("✅ Todas las hojas requeridas están presentes.")
    else:
        print(f"❌ Faltan las siguientes hojas: {hojas_faltantes}")


Funciones para la lectura de los datos y la creación de los correspondientes diccionarios

In [ ]:
def leer_datos(archivo_excel):
    """
    Lee y procesa todas las hojas del archivo Excel de entrada del modelo.

    Esta es la función principal de lectura que coordina el procesamiento de todas
    las hojas del Excel, valida consistencias entre conjuntos, y construye las
    estructuras de datos necesarias para el modelo de optimización.

    El proceso incluye:
    - Lectura secuencial de hojas con headers apropiados
    - Limpieza de datos (eliminación de nulos y columnas vacías)
    - Procesamiento especializado según tipo de hoja
    - Validación de consistencia entre conjuntos relacionados
    - Construcción de conjuntos derivados (U, K, R, T)
    - Incorporación automática de nodos faltantes desde Arcos

    Argumentos:
        archivo_excel (pd.ExcelFile): Objeto ExcelFile de pandas con todas las hojas del modelo

    Salidas:
        dict: Diccionario multinivel con toda la información procesada, conteniendo:
            - 'Conjuntos': Metadatos de tamaños de conjuntos
            - 'Nodos': Parámetros de capacidad por nodo
            - 'Materiales': Características de materiales
            - 'Vehiculos': Configuración de vehículos
            - 'MaterialesNodos': Relaciones material-nodo
            - 'MaterialesTransito': Inventario en tránsito
            - 'VehiculosNodos': Disponibilidad inicial de vehículos
            - 'VehiculosTransito': Vehículos en tránsito
            - 'Arcos': Estructura de red con costos y tiempos
            - 'Demanda': Demanda por material-nodo-tiempo
            - 'CapacidadPdn': Capacidad productiva de nodos
            - 'U': Conjunto de unidades de transporte (K ∪ R)
            - 'K': Conjunto de cabezotes
            - 'R': Conjunto de remolques
            - 'T': Conjunto de períodos de tiempo

    Notas:
        - Las hojas se procesan en orden específico para resolver dependencias
        - Imprime progreso detallado y advertencias durante el procesamiento
        - Nodos referenciados en Arcos se agregan automáticamente a Nodos si faltan
    """

    data = {}
    all_nodes_from_arcos = set() # Set para guardar todos los nodos únicos en la hoja Arcos

    hojas = [
            'Conjuntos',
            'Nodos',
            'Materiales',
            'MaterialesNodos',
            'Vehiculos',
            'VehiculosNodos',
            'Arcos',
            'Demanda',
            'MaterialesTransito',
            'VehiculosTransito',
            'CapacidadPdn'
        ]
    for nombre_hoja in [hoja for hoja in hojas if hoja in archivo_excel.sheet_names]:
        print('')
        print('-'*80)
        print(f'Lectura de la hoja {nombre_hoja}')
        print('')

        # Leer DataFrame con header apropiado
        if nombre_hoja == "Conjuntos":
            df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja, header=0)
        else:
            df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja, header=1)

        # Identificar las columnas llaves de cada hoja
        if nombre_hoja in ["Conjuntos", "Nodos", "Materiales", "Vehiculos"]:
            key_cols = [df.columns[0]]
        elif nombre_hoja in ["MaterialesNodos", "VehiculosNodos", "CapacidadPdn"]:
            key_cols = list(df.columns[:2])
        elif nombre_hoja in ["MaterialesTransito", "VehiculosTransito", "Demanda"]:
            if nombre_hoja == 'Demanda':
                key_cols = [df.columns[1], df.columns[0], df.columns[2]]
            else:
                key_cols = [df.columns[2], df.columns[0], df.columns[1]]
        elif nombre_hoja == "Arcos":
            key_cols = [df.columns[0]]
        else:
            key_cols = []

        # Eliminar las filas donde la(s) llave(s) sean nulas o tengan un espacio en blanco
        if key_cols:
            for col in key_cols:
                if col in df.columns:
                    df = df[df[col].astype(str).str.strip() != '']
            df.dropna(subset=key_cols, inplace=True)


        # Eliminar columnas vacías
        df.dropna(axis=1, how='all', inplace=True)


        # Procesar según el tipo de hoja
        if nombre_hoja in ["Conjuntos", "Nodos", "Materiales", "Vehiculos"]:
            data_hoja = _procesar_hojas_basicas(df, nombre_hoja, data)
            if nombre_hoja == 'Nodos':
                # Asegurar que la información de Nodos se haya procesado antes que Arcos
                data['Nodos'] = data_hoja

        elif nombre_hoja in ["MaterialesNodos", "VehiculosNodos", "CapacidadPdn"]:
            data_hoja = _procesar_hojas_relacion_doble(df, nombre_hoja, data)

        elif nombre_hoja in ["MaterialesTransito", "VehiculosTransito", "Demanda"]:
            data_hoja = _procesar_hojas_transito(df, nombre_hoja, data)

        elif nombre_hoja == "Arcos":
            # Procesar Arcos y recolectar todos los nodos únicos
            data_hoja, nodes_from_arcos = _procesar_arcos(df, data)
            all_nodes_from_arcos.update(nodes_from_arcos)

        else:
            data_hoja = df.to_dict('list')

        data[nombre_hoja] = data_hoja

        # Procesar conjuntos especiales después de leer Vehiculos
        if nombre_hoja == 'Vehiculos':
            _procesar_conjuntos_transporte(data)

        # Procesar conjunto de tiempo después de leer Demanda
        if nombre_hoja == 'Demanda':
            _procesar_conjunto_tiempo(data)

    # Agregar a Nodos a aquellos que están solo definidos en Arcos
    if 'Nodos' in data and all_nodes_from_arcos:
        nodos_from_nodos_sheet = set(data['Nodos'].keys())
        missing_nodes_in_nodos_sheet = all_nodes_from_arcos - nodos_from_nodos_sheet

        if missing_nodes_in_nodos_sheet:
            print('')
            print('-'*80)
            print(f'Adding missing nodes from Arcos to Nodos: {missing_nodes_in_nodos_sheet}')
            # Add missing nodes to the 'Nodos' dictionary with default values
            for node in missing_nodes_in_nodos_sheet:
                data['Nodos'][node] = {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999}
            print('Updated Nodos list:', list(data['Nodos'].keys()))

    return data

Funciones para hojas con parámetros que dependen solo de un conjunto

In [ ]:
def _procesar_hojas_basicas(df, nombre_hoja, data):
    """
    Procesa hojas básicas con parámetros que dependen de un solo conjunto.

    Maneja las hojas: Conjuntos, Nodos, Materiales, Vehiculos. Para Vehiculos,
    realiza procesamiento especial expandiendo cada vehículo según sus condiciones
    de transporte habilitadas.

    Argumentos:
        df (pd.DataFrame): DataFrame con los datos de la hoja
        nombre_hoja (str): Nombre de la hoja siendo procesada
        data (dict): Diccionario acumulativo con datos ya procesados (para validación)

    Salidas:
        dict: Diccionario indexado por el identificador principal (nodo, material, vehículo)
              con subdicionarios conteniendo los parámetros correspondientes

              Para Vehiculos específicamente:
              {
                  'vehiculo_id': {
                      'K': 'cabezote_id',
                      'R': 'remolque_id',
                      'ownership': 'propio' o 'tercerizado',
                      'qvol_v': capacidad_volumen,
                      'qpes_v': capacidad_peso,
                      'cond': 'seco' | 'refrigerado' | 'congelado' | 'ultracongelado'
                  }
              }

    Imprime:
        - Dimensión del dataframe
        - Resultado de verificación de consistencia
        - Lista de llaves únicas encontradas
        - Preview de los primeros registros
        - Advertencias si faltan columnas o datos
    """

    data_hoja = df.set_index(df.columns[0]).to_dict('index')

    if nombre_hoja == 'Conjuntos':
        for conjunto, valores in data_hoja.items():
            print(f'{conjunto}: {valores.get("tamaño", 0)}')
        return data_hoja
    elif nombre_hoja == 'Vehiculos':
        condition_map = {
            'condveh_c1v': 'seco',
            'condveh_c2v': 'refrigerado',
            'condveh_c3v': 'congelado',
            'condveh_c4v': 'ultracongelado'
        }

        # Check if 'ownership' column exists and has a header
        if 'ownership' not in df.columns or df['ownership'].isnull().all():
            # Intento de encontrar columnas con valores no nulos que podrían ser 'ownership'
            potential_ownership_col = None
            for col in df.columns:
                if df[col].notnull().any():
                    # Revisar si hay valores que se ven como "propio" o "tercerizado"
                    if df[col].astype(str).str.contains('propio|tercerizado', case=False).any():
                        potential_ownership_col = col
                        break

            if potential_ownership_col:
                print(f"⚠️ Columna '{potential_ownership_col}' identificada como posible 'ownership'. Renombrando.")
                df.rename(columns={potential_ownership_col: 'ownership'}, inplace=True)
            else:
                print("⚠️ No se pudo identificar la columna 'ownership'. Procediendo sin ella.")


        vehiculos_originales = df.set_index(df.columns[0]).to_dict('index')
        data_hoja = {}

        for key, value in vehiculos_originales.items():
            # Buscar condiciones habilitadas (valor = 1)
            condiciones_habilitadas = []
            for cond, val in value.items():
                if pd.notna(val) and str(cond).startswith('condveh_') and val == 1:
                    condiciones_habilitadas.append(cond)

            # Si no tiene condiciones habilitadas, reportar advertencia
            if not condiciones_habilitadas:
                print(f'⚠️ Vehículo {key} no tiene condiciones habilitadas')
                continue

            # Crear una entrada por cada condición habilitada
            for cond in condiciones_habilitadas:
                # Asegurarse de que la condición exista en el mapa
                if cond in condition_map:
                    data_hoja[key] = {
                        'K': value.get('K'),
                        'R': value.get('R'),
                        'ownership': str(value.get('ownership', '')).strip().lower(), # Ensure it's a string
                        'qvol_v': value.get('qvol_v'),
                        'qpes_v': value.get('qpes_v'),
                        'cond': condition_map[cond]
                    }
                else:
                     print(f'⚠️ Condición "{cond}" para vehículo {key} no encontrada en condition_map')


    # Para otras hojas básicas
    print(f'Dimensión del dataframe extraído: {len(data_hoja)} filas y {len(df.columns)} columnas')

    # Verificar consistencia con conjuntos
    _verificar_consistencia_basica(data_hoja, nombre_hoja, data)

    print(f'{len(data_hoja)} llaves únicas: {list(data_hoja.keys())}')
    print('')
    display(df.head())

    return data_hoja


def _verificar_consistencia_basica(data_hoja, nombre_hoja, data):
    """
    Verifica que el número de elementos procesados coincida con el tamaño declarado.

    Compara la cantidad de elementos únicos encontrados en la hoja procesada con
    el valor declarado en la hoja 'Conjuntos' para detectar inconsistencias.

    Argumentos:
        data_hoja (dict): Diccionario con los datos procesados de la hoja
        nombre_hoja (str): Nombre de la hoja (Nodos, Materiales, Vehiculos)
        data (dict): Diccionario con todas las hojas procesadas hasta el momento

    Imprime:
        - ✅ Si hay consistencia entre tamaño declarado y real
        - ❌ Si hay discrepancia, mostrando ambos valores
        - ⚠️ Si no se puede verificar (falta hoja Conjuntos)
    """

    if 'Conjuntos' not in data:
        print('⚠️ No se puede verificar consistencia: hoja "Conjuntos" no procesada aún')
        return

    conjunto_key = 'Tipos de vehiculos' if nombre_hoja == 'Vehiculos' else nombre_hoja

    if conjunto_key not in data['Conjuntos']:
        print(f'⚠️ Conjunto "{conjunto_key}" no encontrado en hoja Conjuntos')
        return

    valor_declarado = data['Conjuntos'][conjunto_key].get('tamaño', 0)
    valor_real = len(data_hoja)

    if valor_declarado == valor_real:
        print('✅ Verificación de consistencia exitosa')
    else:
        print(f'❌ Error de consistencia: cantidad declarada ({valor_declarado}) vs elementos reales ({valor_real})')

Funciones para hojas con parámetros que dependen de dos conjuntos

In [ ]:
def _procesar_hojas_relacion_doble(df, nombre_hoja, data):
    """
    Procesa hojas con parámetros que dependen de dos conjuntos.

    Maneja relaciones entre dos conjuntos: MaterialesNodos, VehiculosNodos, CapacidadPdn.
    Agrupa datos por pares (conjunto1, conjunto2) y valida consistencia con conjuntos base.

    Argumentos:
        df (pd.DataFrame): DataFrame con las columnas: [conjunto1, conjunto2, parámetro(s)]
        nombre_hoja (str): 'MaterialesNodos', 'VehiculosNodos', o 'CapacidadPdn'
        data (dict): Diccionario con datos procesados previamente

    Salidas:
        dict: Diccionario indexado por tuplas (elemento_conjunto1, elemento_conjunto2)
              con valores según la hoja:
              - MaterialesNodos: {'inv_mn': ..., 'invmax_mn': ..., 'invmin_mn': ...}
              - VehiculosNodos: {'vehi_un': ...}
              - CapacidadPdn: {'qp_mn': ...}

    Imprime:
        - Dimensión del dataframe
        - Resultados de verificación de consistencia
        - Información de llaves únicas y conjuntos involucrados
        - Preview de datos
    """

    key_cols = list(df.columns[:2])

    if nombre_hoja == 'MaterialesNodos':
        value_cols = list(df.columns[2:5])
    else:
        value_cols = list(df.columns[2:3])

    data_hoja = df.groupby(key_cols)[value_cols].apply(lambda x: x.to_dict('records')).to_dict()
    data_hoja = {key: value[0] for key, value in data_hoja.items()}

    print(f'Dimensión del dataframe extraído: {len(data_hoja)} filas y {len(df.columns)} columnas')

    # Verificar consistencia
    _verificar_consistencia_relacion_doble(data_hoja, nombre_hoja, data)

    # Mostrar información de las llaves
    _mostrar_info_llaves_dobles(data_hoja, nombre_hoja)

    print('')
    display(df.head())

    return data_hoja


def _verificar_consistencia_relacion_doble(data_hoja, nombre_hoja, data):
    """
    Verifica que los elementos de ambos conjuntos en la relación estén definidos.

    Valida que todos los elementos referenciados en las llaves (tuplas) existan
    en sus conjuntos base correspondientes (Materiales/U y Nodos).

    Argumentos:
        data_hoja (dict): Diccionario con llaves (elemento1, elemento2)
        nombre_hoja (str): Nombre de la hoja para determinar conjuntos a verificar
        data (dict): Diccionario con todos los conjuntos base

    Imprime:
        - ✅ Si todos los elementos están definidos en sus conjuntos base
        - ❌ Para cada elemento no definido, especificando tipo y conjunto
        - ⚠️ Si faltan conjuntos base por procesar
    """

    set_k = set(data_hoja.keys())
    set_x = list({x for (x, n) in set_k})
    set_n = list({n for (x, n) in set_k})

    flag = False

    # Verificar primer conjunto
    conjunto_ref = 'U' if nombre_hoja == 'VehiculosNodos' else 'Materiales'
    conjunto_ref2 = 'Vehiculos' if nombre_hoja == 'VehiculosNodos' else 'Materiales'

    if conjunto_ref not in data:
        print(f'⚠️ No se puede verificar consistencia: hoja "{conjunto_ref}" no procesada aún')
        flag = True
    else:
        for x in set_x:
            if conjunto_ref == 'U':
                _set = data[conjunto_ref]
            else:
                _set = set(data[conjunto_ref].keys())
            if x not in _set:
                flag = True
                tipo_elemento = 'Unidad de transporte' if conjunto_ref == 'U' else 'Material'
                print(f'❌ Error de consistencia. {tipo_elemento} {x} no definido en la hoja "{conjunto_ref2}"')

    # Verificar nodos
    if 'Nodos' not in data:
        print('⚠️ No se puede verificar consistencia: hoja "Nodos" no procesada aún')
        flag = True
    else:
        for n in set_n:
            if n not in data['Nodos'].keys():
                flag = True
                print(f'❌ Error de consistencia. Nodo {n} no definido en la hoja "Nodos"')

    if not flag:
        print('✅ Verificación de consistencia exitosa')


def _mostrar_info_llaves_dobles(data_hoja, nombre_hoja):
    """
    Muestra información sobre las llaves para hojas de relación doble.

    Argumentos:
        data_hoja (dict): Diccionario con llaves (elemento1, elemento2)
        nombre_hoja (str): Nombre de la hoja para determinar conjuntos a mostrar

    Imprime:
        - Cantidad de llaves únicas
        - Conjuntos involucrados en cada llave
        - Parámetros de cada llave
    """
    set_k = set(data_hoja.keys())
    set_x = list({x for (x, n) in set_k})
    set_n = list({n for (x, n) in set_k})

    conjunto_nombre = nombre_hoja.replace('Nodos', '')

    print(f'{len(set_k)} llaves únicas: {set_k}')
    print(f'    formadas por combinaciones entre los conjuntos {conjunto_nombre} y Nodos')
    print(f'    {conjunto_nombre}: {set_x}')
    print(f'    Nodos: {set_n}')

Funciones para hojas con parámetros que dependen de dos conjuntos y del tiempo

In [ ]:
def _procesar_hojas_transito(df, nombre_hoja, data):
    """
    Procesa hojas con parámetros que dependen de dos conjuntos y tiempo.

    Maneja hojas con dimensión temporal: MaterialesTransito, VehiculosTransito, Demanda.
    Si una hoja está vacía (MaterialesTransito o VehiculosTransito), crea placeholder.

    Argumentos:
        df (pd.DataFrame): DataFrame con columnas [conjunto1, conjunto2, tiempo, valor]
        nombre_hoja (str): 'MaterialesTransito', 'VehiculosTransito', o 'Demanda'
        data (dict): Diccionario con datos ya procesados

    Salidas:
        dict: Diccionario indexado por tuplas (elemento1, elemento2, periodo_t)
              con valores numéricos representando:
              - MaterialesTransito: Inventario en tránsito
              - VehiculosTransito: Vehículos en tránsito
              - Demanda: Cantidad demandada

              Si está vacío, retorna: {(placeholder_elemento1, placeholder_elemento2, placeholder_t): 0}

    Imprime:
        - Dimensión del dataframe
        - Advertencia si está vacío y se crea placeholder
        - Resultados de verificación de consistencia
        - Información de llaves y conjuntos involucrados
    """

    print(f'Dimensión del dataframe extraído: {len(df)} filas y {len(df.columns)} columnas')

    if df.empty and nombre_hoja in ['MaterialesTransito', 'VehiculosTransito']:
        print(f'DataFrame para {nombre_hoja} está vacío. Agregando entrada de placeholder.')
        if nombre_hoja == 'MaterialesTransito':
            print((list(data['Materiales'].keys())[0], list(data['Nodos'].keys())[0], list(data['T'])[0]))
            placeholder_key = (list(data['Materiales'].keys())[0], list(data['Nodos'].keys())[0], list(data['T'])[0])
        elif nombre_hoja == 'VehiculosTransito':
            print((list(data['Vehiculos'].keys())[0], list(data['Nodos'].keys())[0], list(data['T'])[0]))
            placeholder_key = (list(data['Vehiculos'].keys())[0], list(data['Nodos'].keys())[0], list(data['T'])[0])

        data_hoja = {placeholder_key: 0}
        print(f'Placeholder key added: {placeholder_key}')

    else:
        key_cols = [df.columns[0], df.columns[1], df.columns[2]]
        value_col = df.columns[3]
        data_hoja = df.groupby(key_cols)[value_col].first().to_dict()

    # Verificar consistencia
    _verificar_consistencia_transito(data_hoja, nombre_hoja, data)

    # Mostrar información de las llaves
    _mostrar_info_llaves_transito(data_hoja, nombre_hoja)

    print('')
    display(df.head())

    return data_hoja


def _verificar_consistencia_transito(data_hoja, nombre_hoja, data):
    """
    Verifica que el elemento principal de la llave esté definidos.

    Valida que los elementos referenciados en las llaves (tuplas) existan
    en sus conjuntos base correspondientes (Materiales/Vehículos y Nodos).

    Argumentos:
        data_hoja (dict): Diccionario con llaves (elemento1, elemento2)
        nombre_hoja (str): Nombre de la hoja para determinar conjuntos a verificar
        data (dict): Diccionario con todos los conjuntos base

    Imprime:
        - ✅ Si todos los elementos están definidos en sus conjuntos base
        - ❌ Para cada elemento no definido, especificando tipo y conjunto
        - ⚠️ Si faltan conjuntos base por procesar
    """
    set_k = set(data_hoja.keys())
    if not set_k:
        print(f'⚠️ No se puede verificar consistencia para {nombre_hoja}: data_hoja está vacío')
        return

    set_x = list({x for (x, n, t) in set_k})
    set_n = list({n for (x, n, t) in set_k})

    conjunto_ref = 'Vehiculos' if nombre_hoja == 'VehiculosTransito' else 'Materiales'

    flag = False
    if conjunto_ref not in data:
        print(f'⚠️ No se puede verificar consistencia: hoja "{conjunto_ref}" no procesada aún')
        flag = True
    else:
        for x in set_x:
            if conjunto_ref in data and x not in data[conjunto_ref].keys() and not x.startswith('placeholder'): # Excluir placeholder
                flag = True
                tipo_elemento = 'Vehiculo' if conjunto_ref == 'Vehiculos' else 'Material'
                print(f'❌ Error de consistencia. {tipo_elemento} {x} no definido en la hoja "{conjunto_ref}"')

    # Verificar nodos
    if 'Nodos' not in data:
        print('⚠️ No se puede verificar consistencia: hoja "Nodos" no procesada aún')
        flag = True
    else:
        for n in set_n:
            if n not in data['Nodos'].keys():
                flag = True
                print(f'❌ Error de consistencia. Nodo {n} no definido en la hoja "Nodos"')

    if not flag:
        print('✅ Verificación de consistencia exitosa')


def _mostrar_info_llaves_transito(data_hoja, nombre_hoja):
    """
    Muestra información sobre las llaves para hojas de la información de vehículos y materiales en tránsito.

    Argumentos:
        data_hoja (dict): Diccionario con llaves (elemento1, nodo, periodo_tiempo)
        nombre_hoja (str): Nombre de la hoja para determinar conjuntos a mostrar

    Imprime:
        - Cantidad de llaves únicas
        - Conjuntos involucrados en cada llave
        - Parámetros de cada llave
    """

    set_k = set(data_hoja.keys())
    if not set_k:
        print(f'No hay llaves únicas para mostrar en {nombre_hoja}')
        return

    set_x = list({x for (x, n, t) in set_k})
    set_n = list({n for (x, n, t) in set_k})
    set_t = list({t for (x, n, t) in set_k})

    conjunto_nombre = nombre_hoja.replace("Transito", "")

    print(f'{len(set_k)} llaves únicas: {set_k}')
    print(f'    formadas por combinaciones entre los conjuntos {conjunto_nombre} y Nodos')

    if nombre_hoja == 'Demanda':
        print(f'    Materiales: {set_x}')
    else:
        print(f'    {conjunto_nombre}: {set_x}')

    print(f'    Nodos: {set_n}')
    print(f'    Tiempo: {set_t}')

Funciones para hoja con parámetros de los arcos

In [ ]:
def _procesar_arcos(df, data):
    """
    Procesa la hoja de Arcos construyendo la estructura de red del modelo.

    Crea un diccionario jerárquico por arco con información de vehículos compatibles,
    condiciones de transporte, costos, tiempos y capacidades. Convierte tiempos de
    horas a días (redondeando hacia arriba) y aplica correcciones automáticas a
    condiciones mal escritas.

    Argumentos:
        df (pd.DataFrame): DataFrame con columnas requeridas:
            - Columna 0: arc_id (identificador único del arco)
            - 'i': nodo de origen
            - 'j': nodo de destino
            - 'V': identificador del vehículo
            - 'C': condición de transporte (seco, refrigerado, congelado, ultracongelado)
            - Columnas 5-10: parámetros del arco (costo_ijvc, costov_ijvc, tiempo_ijvc,
                            tiempov_ijvc, caparco_ijv, dist_ijv)
        data (dict): Diccionario con datos procesados (para validación de consistencia)

    Salidas:
        tuple: (data_hoja, set_n) donde:
            - data_hoja (dict): Estructura jerárquica de arcos:
                {
                    'arc_id': {
                        'i': nodo_origen,
                        'j': nodo_destino,
                        'vehicles': {
                            'vehiculo_id': {
                                'condicion': {
                                    'costo_ijvc': costo de viaje con carga,
                                    'costov_ijvc': costo de viaje vacío,
                                    'tiempo_ijvc': tiempo de viaje con carga (días),
                                    'tiempov_ijvc': tiempo de viaje vacío (días),
                                    'caparco_ijv': capacidad del arco para ese vehículo,
                                    'dist_ijv': distancia del arco (km)
                                }
                            }
                        }
                    }
                }
            - set_n (set): Conjunto de todos los nodos únicos encontrados en los arcos
                          (usado para agregar nodos faltantes a la hoja Nodos)

    Imprime:
        - Dimensión del dataframe (filas y columnas)
        - ⚠️ Advertencias de correcciones automáticas aplicadas
        - Resultados de verificación de consistencia
        - Número de llaves únicas (arcos distintos)
        - Conjuntos de nodos y vehículos encontrados (ordenados)
        - Preview de las primeras filas del dataframe

    Notas:
        - Tiempos se convierten automáticamente de horas a días: días = ceil(horas / 24)
        - Corrección automática: 'ultracong' → 'ultracongelado' (typo común)
        - Un mismo arco físico puede tener múltiples configuraciones (vehículo-condición)
        - Los features extraídos son siempre las columnas en posiciones 5-10
        - El set_n retornado se usa para validar/completar la lista de nodos
    """

    data_hoja = {}
    set_n = set()
    set_v = set()

    for index, row in df.iterrows():
        arc_id = row[df.columns[0]]
        start_node = row['i']
        end_node = row['j']
        vehicle_id = row['V']
        condition = row['C'].strip().lower()
        features = row[df.columns[5:11]].to_dict()

        set_n.add(start_node)
        set_n.add(end_node)
        set_v.add(vehicle_id)

        # Corrección del hardcode
        if condition == 'ultracong':
            condition = 'ultracongelado'
            print(f'⚠️ Corrigiendo condición "ultracong" a "ultracongelado" para arco {arc_id}')

        # Redondear tiempos a días superiores (convertir de horas a días primero)
        if 'tiempo_ijvc' in features:
            features['tiempo_ijvc'] = math.ceil(features['tiempo_ijvc'] / 24)
        if 'tiempov_ijvc' in features:
            features['tiempov_ijvc'] = math.ceil(features['tiempov_ijvc'] / 24)

        # Construir estructura
        if arc_id not in data_hoja:
            data_hoja[arc_id] = {
                'i': start_node,
                'j': end_node,
                'vehicles': {}
            }

        if vehicle_id not in data_hoja[arc_id]['vehicles']:
            data_hoja[arc_id]['vehicles'][vehicle_id] = {}

        data_hoja[arc_id]['vehicles'][vehicle_id][condition] = features

    print(f'Dimensión del dataframe extraído: {len(df)} filas y {len(df.columns)} columnas')

    # Verificar consistencia
    _verificar_consistencia_arcos(data_hoja, data)

    print(f'{len(data_hoja)} llaves únicas: {list(data_hoja.keys())}')
    print(f'    formadas por combinaciones entre los conjuntos Nodos y Vehiculos')
    print(f'    Nodos: {sorted(list(set_n))}')
    print(f'    Vehiculos: {sorted(list(set_v))}')
    print('')
    display(df.head())

    return data_hoja, set_n


def _verificar_consistencia_arcos(data_hoja, data):
    """
    Verifica que los nodos y vehículos referenciados en arcos estén definidos.

    Valida que todos los nodos (origen y destino) y vehículos mencionados en
    la estructura de arcos existan en sus respectivas hojas base (Nodos y Vehiculos).
    Esta verificación es crítica para evitar referencias a elementos no definidos.

    Argumentos:
        data_hoja (dict): Diccionario de arcos procesados con estructura jerárquica
                         donde cada arco contiene 'i' (origen), 'j' (destino) y
                         'vehicles' (dict de vehículos)
        data (dict): Diccionario con todos los conjuntos base procesados,
                    debe contener las llaves 'Nodos' y 'Vehiculos' para validación

    Imprime:
        - ✅ Si todos los nodos y vehículos están correctamente definidos
        - ❌ Para cada nodo no definido, especificando el nodo y arco problemático
        - ❌ Para cada vehículo no definido, especificando el vehículo y arco problemático
        - ⚠️ Si la hoja 'Nodos' no ha sido procesada aún
        - ⚠️ Si la hoja 'Vehiculos' no ha sido procesada aún

    Valida:
        - Para cada arco (i, j): i ∈ Nodos y j ∈ Nodos
        - Para cada vehículo v en vehicles: v ∈ Vehiculos

    Notas:
        - Esta función se llama automáticamente desde _procesar_arcos()
        - Si falta procesar Nodos o Vehiculos, marca flag=True pero no aborta
        - Los errores de consistencia aquí indican problemas en los datos fuente
        - Es normal ver advertencias si Arcos se procesa antes que Nodos/Vehiculos
    """

    flag = False

    for arc_id, entry in data_hoja.items():
        start_node = entry.get('i')
        end_node = entry.get('j')
        vehicles = entry.get('vehicles', {})

        # Verificar nodos
        if 'Nodos' in data:
            for node in [start_node, end_node]:
                if node not in data['Nodos'].keys():
                    flag = True
                    print(f'❌ Error de consistencia. Nodo {node} no definido en la hoja "Nodos"')
        else:
            print('⚠️ No se puede verificar nodos: hoja "Nodos" no procesada aún')
            flag = True

        # Verificar vehículos
        if 'Vehiculos' in data:
            for vehicle in vehicles:
                if vehicle not in data['Vehiculos'].keys():
                    flag = True
                    print(f'❌ Error de consistencia. Vehiculo {vehicle} no definido en la hoja "Vehiculos"')
        else:
            print('⚠️ No se puede verificar vehículos: hoja "Vehiculos" no procesada aún')
            flag = True

    if not flag:
        print('✅ Verificación de consistencia exitosa')

Funciones especiales para procesar las unidades de transporte y el tiempo

In [ ]:
def _procesar_conjuntos_transporte(data):
    """
    Construye los conjuntos de unidades de transporte (cabezotes y remolques).

    Extrae de la configuración de vehículos las referencias a cabezotes (K)
    y remolques (R), construyendo los conjuntos K, R y su unión U.

    Argumentos:
        data (dict): Diccionario que debe contener 'Vehiculos' con campos 'K' y 'R'

    Modifica:
        data['K'] (set): Conjunto de todos los cabezotes únicos
        data['R'] (set): Conjunto de todos los remolques únicos
        data['U'] (set): Unión de cabezotes y remolques (K ∪ R)

    Imprime:
        - Lista ordenada de cabezotes encontrados
        - Lista ordenada de remolques encontrados
        - Lista ordenada de todas las unidades de transporte (U)
    """

    cabezotes_set = set()
    remolques_set = set()

    for vehiculo_id, vehiculo_data in data['Vehiculos'].items():
        cabezote_id = vehiculo_data.get('K')
        remolque_id = vehiculo_data.get('R')

        if pd.notna(cabezote_id):
            cabezotes_set.add(cabezote_id)

        if pd.notna(remolque_id):
            remolques_set.add(remolque_id)

    U = cabezotes_set.union(remolques_set)
    data['U'] = U
    data['K'] = cabezotes_set
    data['R'] = remolques_set

    print('')
    print('-'*80)
    print(f' Conjunto de cabezotes: {sorted(list(cabezotes_set))}')
    print(f' Conjunto de remolques: {sorted(list(remolques_set))}')
    print(f' Conjunto de unidades de transporte: {sorted(list(U))}')


def _procesar_conjunto_tiempo(data):
    """
    Construye el conjunto de períodos de tiempo a partir de la demanda.

    Extrae todos los períodos únicos presentes en las llaves de la hoja Demanda
    y los ordena cronológicamente para crear el horizonte de planificación.

    Argumentos:
        data (dict): Diccionario que debe contener 'Demanda' con llaves (m, n, t)

    Modifica:
        data['T'] (list): Lista ordenada de períodos de tiempo únicos

    Imprime:
        - Lista de períodos de tiempo encontrados
        - ⚠️ Advertencia si no se encuentra la hoja Demanda

    Notas:
        - Los períodos se extraen de la tercera posición de las llaves (m, n, t)
        - La lista se ordena automáticamente
    """

    if 'Demanda' in data:

        T = sorted({k[2] for k in data['Demanda'].keys()})
        print('')
        print('-'*80)
        print(f'Conjunto de periodos de tiempo: {T}')
        data['T'] = T
    else:
        print('⚠️ No se pudo procesar conjunto de tiempo: hoja "Demanda" no encontrada')

## Funciones para la verificación de factibilidad

In [ ]:
def ejecutar_verificaciones_completas():
    """
    Ejecuta todas las verificaciones de factibilidad del modelo de forma secuencial.

    Función orquestadora que coordina la ejecución de todas las funciones de
    verificación en el orden apropiado, generando un reporte completo del
    estado de los datos.

    Secuencia de ejecución:
        1. mostrar_resumen_datos(): Estadísticas generales
        2. verificar_factibilidad_basica(): Validaciones críticas
        3. verificaciones_detalladas_matrices(): Validaciones de matrices (si es factible)

    Argumentos:
        Ninguno: Usa variables globales del modelo

    Salidas:
        tuple: (es_factible, errores, advertencias) donde:
            - es_factible (bool): True si el modelo puede proceder a optimización
            - errores (list): Lista de errores críticos encontrados
            - advertencias (list): Lista de advertencias no bloqueantes

    Imprime:
        - Separadores visuales entre secciones
        - 🟢 Conclusión positiva si es factible
        - 🔴 Conclusión negativa si hay errores críticos
        - Sugerencia de revisar advertencias para mejorar calidad
    """

    print("=" * 60)

    # Mostrar resumen
    mostrar_resumen_datos()

    # Verificaciones básicas
    print("=" * 60)
    es_factible, errores, advertencias = verificar_factibilidad_basica()

    # Verificaciones detalladas si no hay errores críticos
    print("=" * 60)
    if es_factible:
        verificaciones_detalladas_matrices()

    print("\n" + "=" * 60)
    if es_factible:
        print("🟢 CONCLUSIÓN: El modelo puede proceder a la optimización")
        if advertencias:
            print("   (Revisar advertencias para mejorar la calidad del modelo)")
    else:
        print("🔴 CONCLUSIÓN: Resolver errores críticos antes de optimizar")

    return es_factible, errores, advertencias

def verificar_factibilidad_basica():
    """
    Verifica condiciones básicas de factibilidad del modelo antes de optimizar.

    Ejecuta 12 verificaciones críticas sobre los datos para detectar problemas
    que podrían hacer el modelo infactible o generar soluciones subóptimas.
    Usa las variables globales definidas en la sección de Implementación.

    Verificaciones realizadas:
        1. Inventario inicial suficiente para demanda del primer período
        2. Coherencia de límites de inventario (mín <= máx)
        3. Capacidades de vehículos no negativas
        4. Conectividad de la red (nodos con entrada/salida)
        5. Balance oferta-demanda por material
        6. Compatibilidad material-condición-vehículo
        7. Configuración correcta de flota propia
        8. Disponibilidad inicial de unidades de transporte
        9. Tiempos y costos válidos en arcos
        10. Capacidades operativas de nodos
        11. Factores de conversión de materiales válidos
        12. Capacidades de arcos

    Argumento:
        Ninguna: Usa variables globales (M, N, V, A, T, etc.)

    Salidas:
        tuple: (es_factible, errores, advertencias) donde:
            - es_factible (bool): True si no hay errores críticos
            - errores (list): Lista de strings con errores críticos encontrados
            - advertencias (list): Lista de strings con advertencias no bloqueantes

    Imprime:
        - Progreso de cada verificación numerada
        - Resumen con conteo de errores y advertencias
        - 🔴 Lista detallada de errores críticos (si existen)
        - 🟡 Lista detallada de advertencias (si existen)
        - ✅ Mensaje de éxito si todo está correcto

    Notas:
        - Errores críticos indican problemas que harán el modelo infactible
        - Advertencias son situaciones sospechosas pero no necesariamente problemáticas
        - Se recomienda resolver todos los errores antes de ejecutar optimización
    """

    errores = []
    advertencias = []

    print("\n==== VERIFICACIONES DE FACTIBILIDAD ===\n")

    # 1. VERIFICAR SI LA DEMANDA INICIAL SE PUEDE SUPLIR CON EL INVENTARIO INICIAL Y EN TRANSITO SIN VIOLAR LA COTA INFERIOR
    print("1. Verificando inventario inicial, en tránsito y demanda...")
    for m in M:
        for n in N:
            fact = inv_mn[m][n] + invt_mnt[m][n][T[0]] - dem_mnt[m][n][T[0]]
            if fact < invmin_mn[m][n]:
                errores.append(f"La demanda inicial en el nodo {n} del material {m} ({dem_mnt[m][n][T[0]]}) no puede ser atendida con el inventario inicial ({inv_mn[m][n]}) mas el inventario en tránsito ({invt_mnt[m][n][T[0]]}) sin violar el límite inferior de inventario ({invmin_mn[m][n]})")


    # 2. VERIFICACIONES DE COHERENCIA DE INVENTARIOS
    print("2. Verificando coherencia de límites de inventario...")

    for m in M:
        for n in N:
            inv_min = invmin_mn[m][n]
            inv_max = invmax_mn[m][n]
            if inv_min > inv_max:
                advertencias.append(f"Inventario mínimo > máximo para material {m} en nodo {n}: {inv_min} > {inv_max}")

    # 3. VERIFICACIONES DE CAPACIDADES DE VEHÍCULOS
    print("3. Verificando capacidades de vehículos...")

    for v in V:
        if qvol_v[v] < 0:
            errores.append(f"Capacidad de volumen < 0 para vehículo {v}: {qvol_v[v]}")
        if qpes_v[v] < 0:
            errores.append(f"Capacidad de peso < 0 para vehículo {v}: {qpes_v[v]}")

    # 4. VERIFICACIONES DE CONECTIVIDAD
    print("4. Verificando conectividad de la red...")

    nodos_origen = set()
    nodos_destino = set()
    for (i, j) in A:
        nodos_origen.add(i)
        nodos_destino.add(j)

    nodos_sin_salida = set(N) - nodos_origen
    nodos_sin_entrada = set(N) - nodos_destino

    # Los productores pueden no tener entrada, otros nodos deberían tenerla
    nodos_no_productores_sin_entrada = nodos_sin_entrada - set(N_p)
    if nodos_no_productores_sin_entrada:
        advertencias.append(f"Nodos no productores sin entrada: {nodos_no_productores_sin_entrada}")

    if nodos_sin_salida:
        advertencias.append(f"Nodos sin salida: {nodos_sin_salida}")

    # 5. VERIFICACIONES DE BALANCE OFERTA-DEMANDA
    print("5. Verificando balance oferta-demanda...")

    for m in M:
        # Calcular oferta total
        oferta_total = 0

        # Producción en nodos productores
        for n in N_p:
            if n in qp_mn[m]:
                oferta_total += qp_mn[m][n] * len(T)

        # Inventario inicial
        for n in N:
            oferta_total += inv_mn[m][n]
            # Inventario en tránsito
            for t in T:
                oferta_total += invt_mnt[m][n][t]

        # Calcular demanda total
        demanda_total = 0
        for n in N:
            for t in T:
                demanda_total += dem_mnt[m][n][t]

        if demanda_total > oferta_total:
            errores.append(f"Demanda total > oferta total para material {m}: {demanda_total:.2f} > {oferta_total:.2f}")
        elif demanda_total > 0 and oferta_total > demanda_total * 5:  # Oferta excesiva
            advertencias.append(f"Oferta muy alta vs demanda para material {m}: {oferta_total:.2f} vs {demanda_total:.2f}")
        elif demanda_total == 0:
            advertencias.append(f"Sin demanda para material {m}")

    # 6. VERIFICACIONES DE COMPATIBILIDAD MATERIAL-CONDICIÓN-VEHÍCULO
    print("6. Verificando compatibilidad material-condición-vehículo...")

    for m in M:
        # Encontrar condiciones compatibles con el material
        condiciones_compatibles = []
        for c in C:
            if condmat_cm[c][m] == 1:
                condiciones_compatibles.append(c)

        if not condiciones_compatibles:
            errores.append(f"Material {m} no tiene condiciones de transporte compatibles")
            continue

        # Verificar que existan vehículos para esas condiciones
        vehiculos_compatibles = []
        for v in V:
            for c in condiciones_compatibles:
                if condveh_cv[c][v] == 1:
                    vehiculos_compatibles.append(v)
                    break

        if not vehiculos_compatibles:
            errores.append(f"Material {m} no tiene vehículos compatibles")
        else:
            print(f"   ✓ Material {m}: {len(vehiculos_compatibles)} tipos de vehículos compatibles")

    # 7. VERIFICACIONES DE FLOTA PROPIA
    print("7. Verificando configuración de flota propia...")

    if not V_p:
        advertencias.append("No hay vehículos propios definidos")

    for v in V_p:
        # Verificar composición (debe usar cabezotes y/o remolques)
        usa_cabezotes = any(composic_vu[v][k] == 1 for k in K)
        usa_remolques = any(composic_vu[v][r] == 1 for r in R)

        if not usa_cabezotes and not usa_remolques:
            errores.append(f"Vehículo propio {v} no tiene configuración de cabezotes/remolques")

    # 8. VERIFICACIONES DE DISPONIBILIDAD DE UNIDADES DE TRANSPORTE
    print("8. Verificando disponibilidad inicial de unidades de transporte...")

    for u in U:
        total_disponible = sum(vehi_un[u][n] for n in N) + sum(sum(veht_unt[u][n][t] for t in T) for n in N)
        if total_disponible == 0:
            advertencias.append(f"Sin unidades iniciales disponibles del tipo {u}")


    # 9. VERIFICACIONES DE TIEMPOS Y COSTOS
    print("9. Verificando tiempos y costos...")

    for (i, j) in A:
        # Tiempo de viaje
        tiempo = lt_ij[i][j]
        if tiempo <= 0:
            advertencias.append(f"Tiempo de viaje <= 0 en arco ({i}, {j}): {tiempo}")
        elif tiempo > len(T):
            advertencias.append(f"Tiempo de viaje muy alto en arco ({i}, {j}): {tiempo} > {len(T)} períodos")

        # Costos
        for v in V:
            if v in costotrans_ijv[i][j]:
                costo_carga = costotrans_ijv[i][j][v]
                if costo_carga < 0:
                    errores.append(f"Costo negativo para vehículo {v} en arco ({i}, {j}): {costo_carga}")

            if v in V_p and v in costotransvac_ijv[i][j]:
                costo_vacio = costotransvac_ijv[i][j][v]
                if costo_vacio < 0:
                    errores.append(f"Costo vacío negativo para vehículo {v} en arco ({i}, {j}): {costo_vacio}")

    # 10. VERIFICACIONES DE CAPACIDADES OPERATIVAS DE NODOS
    print("10. Verificando capacidades operativas de nodos...")

    for n in N:
        if qr_n[n] <= 0:
            advertencias.append(f"Capacidad de recepción muy baja en nodo {n}: {qr_n[n]}")
        if qd_n[n] <= 0:
            advertencias.append(f"Capacidad de despacho muy baja en nodo {n}: {qd_n[n]}")

    # 11. VERIFICACIONES DE FACTORES DE CONVERSIÓN
    print("11. Verificando factores de conversión de materiales...")

    for m in M:
        if vol_m[m] <= 0:
            advertencias.append(f"Factor de volumen <= 0 para material {m}: {vol_m[m]}")
        if peso_m[m] <= 0:
            advertencias.append(f"Factor de peso <= 0 para material {m}: {peso_m[m]}")
        if costoinv_m[m] < 0:
            advertencias.append(f"Costo de inventario negativo para material {m}: {costoinv_m[m]}")

    # 12. VERIFICACIÓN DE CAPACIDADES DE ARCOS
    print("12. Verificando capacidades de arcos...")

    arcos_con_restricciones = 0
    for (i, j) in A:
        for v in V:
            if v in caparco_ijv[i][j] and caparco_ijv[i][j][v] < 9999:
                arcos_con_restricciones += 1
                break

    if arcos_con_restricciones == 0:
        advertencias.append("Ningún arco tiene restricciones de capacidad (todos con capacidad muy alta)")

    # RESUMEN
    print("\n=== RESUMEN DE VERIFICACIONES ===")
    print(f"Errores críticos encontrados: {len(errores)}")
    print(f"Advertencias encontradas: {len(advertencias)}")

    if errores:
        print("\n🔴 ERRORES CRÍTICOS (modelo probablemente infactible):")
        for i, error in enumerate(errores, 1):
            print(f"  {i}. {error}")

    if advertencias:
        print("\n🟡 ADVERTENCIAS (revisar pero no bloquean):")
        for i, adv in enumerate(advertencias, 1):
            print(f"  {i}. {adv}")

    if not errores and not advertencias:
        print("\n✅ Todas las verificaciones básicas pasaron correctamente")
    elif not errores:
        print("\n✅ No hay errores críticos - el modelo puede proceder")

    return len(errores) == 0, errores, advertencias


def verificaciones_detalladas_matrices():
    """
    Verifica propiedades de las matrices de compatibilidad y composición.

    Valida que las matrices binarias efectivamente contengan solo valores 0 y 1,
    y que la configuración de vehículos propios sea consistente (cada vehículo
    debe usar exactamente un cabezote y un remolque).

    Argumentos:
        Ninguna: Usa variables globales (condmat_cm, condveh_cv, composic_vu, C, M, V, V_p, U, K, R)

    Imprime:
        - ⚠️ Para cada valor que no sea 0 o 1 en matrices binarias
        - ⚠️ Para vehículos propios que no usen exactamente 1 cabezote y 1 remolque
        - Confirmación de finalización de verificaciones

    Verifica:
        - condmat_cm[c][m] ∈ {0, 1} para todo c ∈ C, m ∈ M
        - condveh_cv[c][v] ∈ {0, 1} para todo c ∈ C, v ∈ V
        - composic_vu[v][u] ∈ {0, 1} para todo v ∈ V_p, u ∈ U
        - Σ_k composic_vu[v][k] = 1 para todo v ∈ V_p
        - Σ_r composic_vu[v][r] = 1 para todo v ∈ V_p
    """

    print("\n=== VERIFICACIONES DETALLADAS DE MATRICES ===")

    # Verificar matrices binarias
    print("1. Verificando que las matrices sean binarias...")

    # condmat_cm
    for c in C:
        for m in M:
            if condmat_cm[c][m] not in [0, 1]:
                print(f"⚠️ condmat_cm[{c}][{m}] = {condmat_cm[c][m]} (debería ser 0 o 1)")

    # condveh_cv
    for c in C:
        for v in V:
            if condveh_cv[c][v] not in [0, 1]:
                print(f"⚠️ condveh_cv[{c}][{v}] = {condveh_cv[c][v]} (debería ser 0 o 1)")

    # composic_vu
    for v in V_p:
        for u in U:
            if composic_vu[v][u] not in [0, 1]:
                print(f"⚠️ composic_vu[{v}][{u}] = {composic_vu[v][u]} (debería ser 0 o 1)")

    print("2. Verificando consistencia de configuraciones...")

    # Cada vehículo propio debe usar exactamente un cabezote y un remolque
    for v in V_p:
        cabezotes_usados = sum(composic_vu[v][k] for k in K)
        remolques_usados = sum(composic_vu[v][r] for r in R)

        if cabezotes_usados != 1:
            print(f"⚠️ Vehículo {v} usa {cabezotes_usados} cabezotes (debería ser 1)")
        if remolques_usados != 1:
            print(f"⚠️ Vehículo {v} usa {remolques_usados} remolques (debería ser 1)")

def mostrar_resumen_datos():
    """
    Muestra un resumen estadístico de los datos del problema.

    Presenta información agregada sobre la dimensión del problema y balance
    oferta-demanda para facilitar comprensión del alcance del modelo.

    Argumentos:
        Ninguna: Usa variables globales (N, N_p, M, V, V_p, C, K, R, T, A, etc.)

    Imprime:
        - Cantidad de elementos por conjunto (nodos, materiales, vehículos, etc.)
        - Proporción de vehículos propios vs tercerizados
        - Nodos productores identificados
        - Condiciones de transporte disponibles
        - Por cada material:
            * Demanda total en el horizonte de planificación
            * Capacidad productiva total disponible

    Notas:
        - Útil para validar que los datos se leyeron correctamente
        - Permite detectar rápidamente desbalances oferta-demanda
    """

    print("\n=== RESUMEN DE DATOS DEL PROBLEMA ===")
    print(f"Nodos: {len(N)} (Productores: {len(N_p)})")
    print(f"Materiales: {len(M)}")
    print(f"Vehículos: {len(V)} (Propios: {len(V_p)}, Tercerizados: {len(V) - len(V_p)})")
    print(f"Condiciones: {len(C)}")
    print(f"Cabezotes: {len(K)}, Remolques: {len(R)}")
    print(f"Períodos: {len(T)}")
    print(f"Arcos: {len(A)}")

    print("\nNodos productores:", N_p)
    print("Condiciones:", C)

    # Resumen de demanda y capacidad productiva
    for m in M:
        demanda_total = sum(dem_mnt[m][n][t] for n in N for t in T)
        print(f"\nDemanda total del material {m} en el horizonte: {demanda_total:.2f}")
        capacidad_total = sum(qp_mn[m][n] * len(T) for n in N_p if n in qp_mn[m].keys())
        print(f"Capacidad productiva total del material {m}: {capacidad_total:.2f}")


## Funciones para los resultados del modelo

Función para extraer los valores de las variables como diccionarios

In [ ]:
def extraer_valores_variables(model):
    """
    Extrae y organiza los valores de las variables de decisión del modelo resuelto.

    Parsea los nombres de variables de PuLP y construye diccionarios indexados
    por las tuplas originales de índices, facilitando el acceso a los resultados.

    Argumentoss:
        model (pulp.LpProblem): Modelo PuLP ya resuelto

    Salidas:
        dict: Diccionario con 7 sub-diccionarios, uno por tipo de variable:
            {
                'X': {(i, j, m, t): valor, ...},     # Flujo de materiales
                'YP': {(i, j, v, t): valor, ...},    # Viajes vehículos propios
                'YT': {(i, j, v, t): valor, ...},    # Viajes vehículos tercerizados
                'W': {(i, j, v, t): valor, ...},     # Viajes vacíos
                'B': {(i, j, u, t): valor, ...},     # Movimiento unidades transporte
                'I': {(m, n, t): valor, ...},        # Niveles de inventario
                'S': {(u, n, t): valor, ...}         # Stock de unidades
            }

    Notas:
        - Solo incluye variables con valor no nulo
        - Parsea nombres de variables tipo 'X_(i,_j,_m,_t)' automáticamente
        - Los índices de tiempo se convierten a enteros
        - Útil para análisis post-optimización y exportación de resultados
    """

    variable_values = {
        'X': {},
        'YP': {},
        'YT': {},
        'W': {},
        'B': {},
        'I': {},
        'S': {}
    }

    for v in model.variables():
        if v.varValue is not None: # and v.varValue != 0: # Incluir valores cero si es necesario
            if v.name.startswith('X_'):
                # Parse X_(i,j,m,t)
                parts = v.name[2:].strip('()').split(',_')
                if len(parts) == 4:
                    key = (parts[0].strip("'"), parts[1].strip("'"), parts[2].strip("'"), int(parts[3].strip("'")))
                    variable_values['X'][key] = v.varValue
            elif v.name.startswith('YP_'):
                # Parse YP_(i,j,v,t)
                parts = v.name[3:].strip('()').split(',_')
                if len(parts) == 4:
                    key = (parts[0].strip("'"), parts[1].strip("'"), parts[2].strip("'"), int(parts[3].strip("'")))
                    variable_values['YP'][key] = v.varValue
            elif v.name.startswith('YT_'):
                # Parse YT_(i,j,v,t)
                parts = v.name[3:].strip('()').split(',_')
                if len(parts) == 4:
                    key = (parts[0].strip("'"), parts[1].strip("'"), parts[2].strip("'"), int(parts[3].strip("'")))
                    variable_values['YT'][key] = v.varValue
            elif v.name.startswith('W_'):
                # Parse W_(i,j,v,t)
                parts = v.name[2:].strip('()').split(',_')
                if len(parts) == 4:
                    key = (parts[0].strip("'"), parts[1].strip("'"), parts[2].strip("'"), int(parts[3].strip("'")))
                    variable_values['W'][key] = v.varValue
            elif v.name.startswith('B_'):
                # Parse B_(i,j,u,t)
                parts = v.name[2:].strip('()').split(',_')
                if len(parts) == 4:
                    key = (parts[0].strip("'"), parts[1].strip("'"), parts[2].strip("'"), int(parts[3].strip("'")))
                    variable_values['B'][key] = v.varValue
            elif v.name.startswith('I_'):
                # Parse I_(m,n,t)
                parts = v.name[2:].strip('()').split(',_')
                if len(parts) == 3:
                    key = (parts[0].strip("'"), parts[1].strip("'"), int(parts[2].strip("'")))
                    variable_values['I'][key] = v.varValue
            elif v.name.startswith('S_'):
                # Parse S_(u,n,t)
                parts = v.name[2:].strip('()').split(',_')
                if len(parts) == 3:
                    key = (parts[0].strip("'"), parts[1].strip("'"), int(parts[2].strip("'")))
                    variable_values['S'][key] = v.varValue

    return variable_values

Función para calcular los indicadores

In [ ]:
def indicadores(model):
    """
    Calcula indicadores clave de desempeño (KPIs) del modelo optimizado.

    Genera métricas operacionales y económicas para evaluar la calidad de la
    solución y el desempeño del sistema de transporte.

    Argumentos:
        model (pulp.LpProblem): Modelo PuLP resuelto con status óptimo

    Salidas:
        dict: Diccionario con los siguientes KPIs:
            - 'FO' (float): Valor de la función objetivo (costo total)
            - 'costo_transporte' (float): Costo total de transporte (propio + tercerizado)
            - 'costo_almacenamiento' (float): Costo total de almacenamiento
            - 'total_viajes' (float): Total de viajes (tercerizados + propios con carga + propios vacíos)
            - 'utiliz_peso' (dict): Utilización de capacidad en peso por arco {(i,j): ratio}
            - 'utiliz_volumen' (dict): Utilización de capacidad en volumen por arco {(i,j): ratio}
            - 'pesoexc' (float): Capacidad excedente total en peso
            - 'volexc' (float): Capacidad excedente total en volumen
            - 'viajesterc' (float): Proporción de viajes tercerizados [0-1]
            - 'viajesprop' (float): Proporción de viajes con flota propia [0-1]
            - 'gastospot' (float): Proporción del gasto en tercerización [0-1]
            - 'kmvacio' (float): Kilómetros totales recorridos en vacío
            - 'propkmvacio' (float): Proporción de km en vacío [0-1]
            - 'usotipo' (dict): Proporción de uso por tipo de vehículo propio {v: ratio}

    Notas:
        - Las utilizaciones se calculan considerando compatibilidad material-condición
        - Ratios cercanos a 1.0 indican alta utilización de capacidad
        - Km en vacío solo aplica para flota propia (reposicionamiento)
        - Usa variables globales de parámetros del modelo
    """

    variable_values = extraer_valores_variables(model)
    X_values = variable_values['X']
    YP_values = variable_values['YP']
    YT_values = variable_values['YT']
    W_values = variable_values['W']
    B_values = variable_values['B']
    I_values = variable_values['I']
    S_values = variable_values['S']

    kpis = {}

    # Valor de la FO
    kpis['FO'] = pl.value(model.objective)

    # Costo de transporte (propio + tercerizado)
    costo_transporte = pl.lpSum(costotrans_ijv.get(i, {}).get(j, {}).get(v, 0)*YP_values.get((i, j, v, t), 0) + costotransvac_ijv.get(i, {}).get(j, {}).get(v, 0)*W_values.get((i, j, v, t), 0) for i in N for j in N if (i, j) in A for t in T for v in V_p) + pl.lpSum(costotrans_ijv.get(i, {}).get(j, {}).get(v, 0)*YT_values.get((i, j, v, t), 0) for i in N for j in N if (i, j) in A for t in T for v in V if v not in V_p)
    kpis['costo_transporte'] = pl.value(costo_transporte)

    # Costo de almacenamiento
    costo_almacenamiento = pl.lpSum(costoinv_m.get(m, 0)*I_values.get((m, n, t), 0) for m in M for n in N for t in T)
    kpis['costo_almacenamiento'] = pl.value(costo_almacenamiento)

    # Total de viajes (tercerizados + propios con carga + propios vacíos)
    total_viajes = pl.lpSum(YP_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V_p) + \
                   pl.lpSum(YT_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V if v not in V_p) + \
                   pl.lpSum(W_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V_p)
    kpis['total_viajes'] = pl.value(total_viajes)


    # Utilización de capacidad en peso por arco
    p_utiliz = {}
    for (i, j) in A:
        num = pl.lpSum(condmat_cm.get(c, {}).get(m, 0) * peso_m.get(m, 0) * X_values.get((i, j, m, t), 0) for t in T for c in C for m in M)
        den = pl.lpSum(qpes_v.get(v, 0) * condveh_cv.get(c, {}).get(v, 0) * (YP_values.get((i, j, v, t), 0) + YT_values.get((i, j, v, t), 0)) for t in T for c in C for v in V)
        if pl.value(den) != 0:
            p_utiliz[(i, j)] = num / den
    kpis['utiliz_peso'] = p_utiliz

    # Utilización de capacidad en volumen por arco
    v_utiliz = {}
    for (i, j) in A:
        num = pl.lpSum(condmat_cm.get(c, {}).get(m, 0) * vol_m.get(m, 0) * X_values.get((i, j, m, t), 0) for t in T for c in C for m in M)
        den = pl.lpSum(qvol_v.get(v, 0) * condveh_cv.get(c, {}).get(v, 0) * (YP_values.get((i, j, v, t), 0) + YT_values.get((i, j, v, t), 0)) for t in T for c in C for v in V)
        if pl.value(den) != 0:
            v_utiliz[(i, j)] = num / den
    kpis['utiliz_volumen'] = v_utiliz


    # Capacidad excedente total en peso
    pesoexc = (pl.lpSum(qpes_v.get(v, 0) * condveh_cv.get(c, {}).get(v, 0) * (YP_values.get((i, j, v, t), 0) + YT_values.get((i, j, v, t), 0)) for (i, j) in A for t in T for c in C for v in V) -
            pl.lpSum(condmat_cm.get(c, {}).get(m, 0) * peso_m.get(m, 0) * X_values.get((i, j, m, t), 0) for (i, j) in A for t in T for c in C for m in M))
    kpis['pesoexc'] = pl.value(pesoexc)

    # Capacidad excedente total en volumen
    volexc = (pl.lpSum(qvol_v.get(v, 0) * condveh_cv.get(c, {}).get(v, 0) * (YP_values.get((i, j, v, t), 0) + YT_values.get((i, j, v, t), 0)) for (i, j) in A for t in T for c in C for v in V) -
            pl.lpSum(condmat_cm.get(c, {}).get(m, 0) * vol_m.get(m, 0) * X_values.get((i, j, m, t), 0) for (i, j) in A for t in T for c in C for m in M))
    kpis['volexc'] = pl.value(volexc)

    # Proporción de viajes tercerizados
    num = pl.lpSum(YT_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V if v not in V_p)
    den = pl.lpSum(YT_values.get((i, j, v, t), 0) + YP_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V)
    viajesterc = num / den if pl.value(den) != 0 else 0
    kpis['viajesterc'] = viajesterc

    # Proporción de viajes propios
    num = pl.lpSum(YP_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V_p)
    den = pl.lpSum(YT_values.get((i, j, v, t), 0) + YP_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V)
    viajesprop = num / den if pl.value(den) != 0 else 0
    kpis['viajesprop'] = viajesprop

    # Proporción del gasto en transporte tercerizado
    num = pl.lpSum(costotrans_ijv.get(i, {}).get(j, {}).get(v, 0)*YT_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V if v not in V_p)
    den = pl.lpSum(costotrans_ijv.get(i, {}).get(j, {}).get(v, 0)*(YT_values.get((i, j, v, t), 0) + YP_values.get((i, j, v, t), 0)) for (i, j) in A for t in T for v in V)
    gastospot = num / den if pl.value(den) != 0 else 0
    kpis['gastospot'] = gastospot

    # Km en vacío
    kmvacio = pl.lpSum(dist_ij.get(i, {}).get(j, 0)*W_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V_p)
    kpis['kmvacio'] = pl.value(kmvacio)

    # Proporción de km en vacío
    num = pl.lpSum(dist_ij.get(i, {}).get(j, 0)*W_values.get((i, j, v, t), 0) for (i, j) in A for t in T for v in V_p)
    den = pl.lpSum(dist_ij.get(i, {}).get(j, 0)*(YP_values.get((i, j, v, t), 0)+W_values.get((i, j, v, t), 0)) for (i, j) in A for t in T for v in V_p)
    propkmvacio = num / den if pl.value(den) != 0 else 0
    kpis['propkmvacio'] = propkmvacio


    # Proporción por tipo de vehículo propio
    usotipo = {}
    for v in V_p:
        num = pl.lpSum(YP_values.get((i, j, v, t), 0) for (i, j) in A for t in T)
        den = pl.lpSum(YP_values.get((i, j, w, t), 0) for (i, j) in A for t in T for w in V_p)
        if pl.value(den) != 0:
            usotipo[v] = num / den
    kpis['usotipo'] = usotipo

    return kpis

Función para extraer los resultados como archivo de excel

In [ ]:
def exportar_resultados_excel(model, archivo='resultados_transporte.xlsx'):
    """
    Exporta los resultados del modelo optimizado a un archivo Excel multi-hoja.

    Genera un archivo Excel estructurado con documentación, resultados detallados
    por variable, y resumen de KPIs. Ideal para análisis y presentación de resultados.

    Estructura del archivo generado:
        - 00_DOCUMENTACION: Guía completa del contenido y estructura
        - X_FlujoMaterial: Flujos de materiales por arco y período
        - YP_VehiculosPropios: Viajes realizados por vehículos propios
        - YT_VehiculosTercerizados: Viajes realizados por vehículos tercerizados
        - W_VehiculosVacios: Reposicionamientos de flota propia
        - B_UnidadesTransporte: Movimientos de cabezotes y remolques
        - I_Inventarios: Niveles de inventario de materiales por nodo y periodo
        - S_StockUnidades: Stock de unidades de transporte
        - RESUMEN_KPIs: Indicadores clave con uso por vehículo y utilización por arco

    Argumentos:
        model (pulp.LpProblem): Modelo PuLP resuelto
        archivo (str, optional): Nombre del archivo de salida. Default: 'resultados_transporte.xlsx'

    Salidas:
        Archivo de Excel con resultados optimizados

    Características:
        - Solo incluye valores significativos (>0.003 para flujos, !=0 para el resto)
        - Calcula métricas derivadas (peso total, volumen total, km totales, costos)
        - Hoja de documentación formateada con colores y estilos profesionales
        - Datos ordenados por período, origen, destino para facilitar análisis
        - Integra KPIs calculados automáticamente

    Imprime:
        - Confirmación con ruta del archivo exportado

    Notas:
        - Requiere openpyxl para generación del Excel
        - Usa zona horaria 'America/Bogota' para timestamp
        - Variables globales usadas: peso_m, vol_m, qpes_v, qvol_v, dist_ij, costos
    """

    # Extraer valores de variables del modelo
    variable_values = extraer_valores_variables(model)
    X_values = variable_values['X']
    YP_values = variable_values['YP']
    YT_values = variable_values['YT']
    W_values = variable_values['W']
    B_values = variable_values['B']
    I_values = variable_values['I']
    S_values = variable_values['S']

    # Crear archivo Excel con múltiples hojas
    with pd.ExcelWriter(archivo, engine='openpyxl') as writer:

        # === HOJA INTRODUCTORIA ===

        # Información general del archivo
        info_general = [
            ['ARCHIVO DE RESULTADOS - MODELO DE TRANSPORTE', ''],
            ['', ''],
            ['Descripción:', 'Este archivo contiene los resultados optimizados del modelo de transporte'],
            ['Fecha de generación:', pd.Timestamp.now(tz='America/Bogota').strftime('%Y-%m-%d %H:%M:%S %Z')],
            ['', ''],
            ['ESTRUCTURA DEL ARCHIVO:', ''],
            ['', ''],
            ['HOJA', 'DESCRIPCIÓN'],
            ['X_FlujoMaterial', 'Cantidades de material transportado entre nodos por periodo'],
            ['YP_VehiculosPropios', 'Viajes realizados por vehículos propios'],
            ['YT_VehiculosTercerizados', 'Viajes realizados por vehículos tercerizados'],
            ['W_VehiculosVacios', 'Movimientos de vehículos propios sin carga (reposicionamiento)'],
            ['B_UnidadesTransporte', 'Movimiento de unidades de transporte especializadas'],
            ['I_Inventarios', 'Niveles de inventario de materiales por nodo y periodo'],
            ['S_StockUnidades', 'Stock de unidades de transporte por nodo y periodo'],
            ['RESUMEN_KPIs', 'Indicadores clave con uso por vehículo y utilización por arco'],
            ['', ''],
            ['COLUMNAS POR HOJA:', ''],
            ['', ''],
        ]

        # Descripción de columnas por hoja
        columnas_info = [
            ['X_FlujoMaterial:', ''],
            ['  • Origen/Destino', 'Nodos de origen y destino del flujo'],
            ['  • Material', 'Tipo de material transportado'],
            ['  • Periodo', 'Periodo de tiempo'],
            ['  • Cantidad', 'Cantidad de material transportado'],
            ['  • Peso_Total', 'Peso total = Cantidad × Factor de peso del material'],
            ['  • Volumen_Total', 'Volumen total = Cantidad × Factor de volumen del material'],
            ['', ''],
            ['YP_VehiculosPropios:', ''],
            ['  • Origen/Destino', 'Arco de la red por donde transita el vehículo'],
            ['  • Vehiculo', 'Tipo de vehículo propio utilizado'],
            ['  • Periodo', 'Periodo de tiempo'],
            ['  • Num_Viajes', 'Número de viajes realizados'],
            ['  • Capacidad_Peso/Volumen', 'Capacidades del tipo de vehículo'],
            ['  • Distancia', 'Distancia del arco en km'],
            ['  • Km_Total', 'Kilómetros totales = Num_Viajes × Distancia'],
            ['  • Costo_Total', 'Costo total = Num_Viajes × Costo unitario de recorrer el arco con ese tipo de vehículo'],
            ['', ''],
            ['YT_VehiculosTercerizados:', ''],
            ['  • Origen/Destino', 'Arco de la red por donde transita el vehículo'],
            ['  • Vehiculo', 'Tipo de vehículo tercerizado utilizado'],
            ['  • Periodo', 'Periodo de tiempo'],
            ['  • Num_Viajes', 'Número de viajes realizados'],
            ['  • Capacidad_Peso/Volumen', 'Capacidades del tipo de vehículo'],
            ['  • Distancia', 'Distancia del arco en km'],
            ['  • Km_Total', 'Kilómetros totales = Num_Viajes × Distancia'],
            ['  • Costo_Total', 'Costo total = Num_Viajes × Costo unitario de recorrer el arco con ese tipo de vehículo'],
            ['', ''],
            ['W_VehiculosVacios:', ''],
            ['  • Origen/Destino', 'Arco de reposicionamiento'],
            ['  • Vehiculo', 'Tipo de vehículo propio que se reposiciona'],
            ['  • Periodo', 'Periodo de tiempo'],
            ['  • Num_Viajes_Vacio', 'Número de viajes sin carga realizados'],
            ['  • Km_Vacio', 'Kilómetros totales en vacío = Num_Viajes × Distancia'],
            ['  • Costo_Total', 'Costo total = Num_Viajes × Costo unitario de recorrer el arco con ese tipo de vehículo en vacío'],
            ['', ''],
            ['B_UnidadesTransporte:', ''],
            ['  • Origen/Destino', 'Arco por donde transita la unidad'],
            ['  • Unidad_Transporte', 'Tipo de unidad de transporte'],
            ['  • Periodo', 'Periodo de tiempo'],
            ['  • Cantidad', 'Cantidad que transita'],
            ['', ''],
            ['I_Inventarios:', ''],
            ['  • Material', 'Tipo de material almacenado'],
            ['  • Nodo', 'Ubicación del inventario'],
            ['  • Periodo', 'Periodo de tiempo'],
            ['  • Nivel_Inventario', 'Cantidad almacenada'],
            ['', ''],
            ['S_StockUnidades:', ''],
            ['  • Unidad_Transporte', 'Tipo de unidad de transporte'],
            ['  • Nodo', 'Ubicación del stock'],
            ['  • Periodo', 'Periodo de tiempo'],
            ['  • Cantidad', 'Cantidad disponible'],
            ['', ''],
            ['RESUMEN_KPIs:', ''],
            ['  • Costo_Total', 'Valor de la función objetivo (costo total de operación)'],
            ['  • Costo_Transporte', 'Costo total de transporte (propio + tercerizado)'],
            ['  • Costo_Almacenamiento', 'Costo total de almacenamiento'],
            ['  • Total_Viajes', 'Número total de viajes (propios con carga, vacíos y tercerizados)'],
            ['  • Exceso_Capacidad_Peso/Volumen', 'Capacidad no utilizada en peso y volumen'],
            ['  • Viajes_Tercerizados/Propios', 'Proporción de viajes en vehículos propios y tercerizados'],
            ['  • Gasto_Tercerización', 'Porcentaje del gasto en tercerización'],
            ['  • Km_en_Vacío', 'Kilómetros de reposicionamiento'],
            ['  • Uso_por_Tipo_Vehículo', 'Distribución del uso de flota propia'],
            ['  • Utilización_por_Arco', 'Eficiencia de uso de capacidad por ruta'],
            ['', ''],
            ['NOTAS IMPORTANTES:', ''],
            ['• Solo se muestran valores > 0', 'Variables con valor cero se omiten por claridad'],
            ['• Todos los periodos en una hoja', 'Usar filtros de Excel para analizar por periodo'],
            ['• Costos en unidades monetarias', 'Según parámetros del modelo'],
            ['• Distancias en km', 'Según matriz de distancias'],
            ['• Pesos en toneladas, volúmenes en m³', 'Según especificaciones del material'],
            ['• Para análisis detallado', 'Usar tablas dinámicas y filtros de Excel'],
            ['', ''],
        ]

        # Combinar toda la información
        documentacion_completa = info_general + columnas_info

        df_doc = pd.DataFrame(documentacion_completa, columns=['Campo', 'Descripción'])
        df_doc.to_excel(writer, sheet_name='00_DOCUMENTACION', index=False)

        # Formatear la hoja de documentación
        worksheet_doc = writer.sheets['00_DOCUMENTACION']

        # Ajustar anchos de columna
        worksheet_doc.column_dimensions['A'].width = 35
        worksheet_doc.column_dimensions['B'].width = 70

        # Aplicar formato a títulos principales
        from openpyxl.styles import Font, PatternFill, Alignment

        title_font = Font(bold=True, size=14, color='FFFFFF')
        title_fill = PatternFill(start_color='2F5597', end_color='2F5597', fill_type='solid')
        section_font = Font(bold=True, size=12, color='2F5597')

        # Formatear título principal
        worksheet_doc['A1'].font = title_font
        worksheet_doc['A1'].fill = title_fill
        worksheet_doc['B1'].font = title_font
        worksheet_doc['B1'].fill = title_fill
        worksheet_doc['A1'].alignment = Alignment(horizontal='center')
        worksheet_doc['B1'].alignment = Alignment(horizontal='center')


        # Formatear secciones principales
        section_rows = [7, 19, 21, 29, 39, 49, 57, 63, 69, 75, 84]
        for row in section_rows:
            if row <= worksheet_doc.max_row:
                cell = worksheet_doc[f'A{row}']
                cell.font = section_font



        # === FLUJOS DE MATERIAL (X) ===
        data_material = []
        for (i, j, m, t), value in X_values.items():
            if value > 0.003:
                data_material.append({
                    'Origen': i,
                    'Destino': j,
                    'Material': m,
                    'Periodo': t,
                    'Cantidad': value,
                    'Peso_Total': value * peso_m.get(m, 0),
                    'Volumen_Total': value * vol_m.get(m, 0)
                })

        df_material = pd.DataFrame(data_material)
        if not df_material.empty:
            df_material = df_material.sort_values(['Periodo', 'Origen', 'Destino', 'Material'])
            df_material.to_excel(writer, sheet_name='X_FlujoMaterial', index=False)

        # === VEHÍCULOS PROPIOS (YP) ===
        data_vehiculos_propios = []
        for (i, j, v, t), value in YP_values.items():
            if value != 0:
                data_vehiculos_propios.append({
                    'Origen': i,
                    'Destino': j,
                    'Vehiculo': v,
                    'Periodo': t,
                    'Num_Viajes': value,
                    'Capacidad_Peso': qpes_v.get(v, 0),
                    'Capacidad_Volumen': qvol_v.get(v, 0),
                    'Distancia': dist_ij.get(i, {}).get(j, 0),
                    'Km_Total': value * dist_ij.get(i, {}).get(j, 0),
                    'Costo_Total': value * costotrans_ijv.get(i, {}).get(j, {}).get(v, 0)
                })

        df_veh_propios = pd.DataFrame(data_vehiculos_propios)
        if not df_veh_propios.empty:
            df_veh_propios = df_veh_propios.sort_values(['Periodo', 'Origen', 'Destino', 'Vehiculo'])
            df_veh_propios.to_excel(writer, sheet_name='YP_VehiculosPropios', index=False)

        # === VEHÍCULOS TERCERIZADOS (YT) ===
        data_vehiculos_terc = []
        for (i, j, v, t), value in YT_values.items():
            if value != 0:
                data_vehiculos_terc.append({
                    'Origen': i,
                    'Destino': j,
                    'Vehiculo': v,
                    'Periodo': t,
                    'Num_Viajes': value,
                    'Capacidad_Peso': qpes_v.get(v, 0),
                    'Capacidad_Volumen': qvol_v.get(v, 0),
                    'Distancia': dist_ij.get(i, {}).get(j, 0),
                    'Km_Total': value * dist_ij.get(i, {}).get(j, 0),
                    'Costo_Total': value * costotrans_ijv.get(i, {}).get(j, {}).get(v, 0)
                })

        df_veh_terc = pd.DataFrame(data_vehiculos_terc)
        if not df_veh_terc.empty:
            df_veh_terc = df_veh_terc.sort_values(['Periodo', 'Origen', 'Destino', 'Vehiculo'])
            df_veh_terc.to_excel(writer, sheet_name='YT_VehiculosTercerizados', index=False)

        # === VEHÍCULOS VACÍOS (W) ===
        data_vacios = []
        for (i, j, v, t), value in W_values.items():
            if value != 0:
                data_vacios.append({
                    'Origen': i,
                    'Destino': j,
                    'Vehiculo': v,
                    'Periodo': t,
                    'Num_Viajes_Vacio': value,
                    'Km_Vacio': value * dist_ij.get(i, {}).get(j, 0),
                    'Costo_Total': value * costotransvac_ijv.get(i, {}).get(j, {}).get(v, 0)
                })

        df_vacios = pd.DataFrame(data_vacios)
        if not df_vacios.empty:
            df_vacios = df_vacios.sort_values(['Periodo', 'Origen', 'Destino', 'Vehiculo'])
            df_vacios.to_excel(writer, sheet_name='W_VehiculosVacios', index=False)

        # === UNIDADES DE TRANSPORTE (B) ===
        data_unidades_transporte = []
        for (i, j, u, t), value in B_values.items():
            if value != 0:
                data_unidades_transporte.append({
                    'Origen': i,
                    'Destino': j,
                    'Unidad_Transporte': u,
                    'Periodo': t,
                    'Cantidad': value
                })

        df_unidades_transporte = pd.DataFrame(data_unidades_transporte)
        if not df_unidades_transporte.empty:
            df_unidades_transporte = df_unidades_transporte.sort_values(['Periodo', 'Origen', 'Destino', 'Unidad_Transporte'])
            df_unidades_transporte.to_excel(writer, sheet_name='B_UnidadesTransporte', index=False)

        # === INVENTARIOS (I) ===
        data_inventarios = []
        for (m, n, t), value in I_values.items():
            if value != 0:
                data_inventarios.append({
                    'Material': m,
                    'Nodo': n,
                    'Periodo': t,
                    'Nivel_Inventario': value
                })

        df_inventarios = pd.DataFrame(data_inventarios)
        if not df_inventarios.empty:
            df_inventarios = df_inventarios.sort_values(['Periodo', 'Nodo', 'Material'])
            df_inventarios.to_excel(writer, sheet_name='I_Inventarios', index=False)

        # === STOCK DE UNIDADES (S) ===
        data_stock_unidades = []
        for (u, n, t), value in S_values.items():
            if value != 0:
                data_stock_unidades.append({
                    'Unidad_Transporte': u,
                    'Nodo': n,
                    'Periodo': t,
                    'Cantidad': value
                })

        df_stock_unidades = pd.DataFrame(data_stock_unidades)
        if not df_stock_unidades.empty:
            df_stock_unidades = df_stock_unidades.sort_values(['Periodo', 'Nodo', 'Unidad_Transporte'])
            df_stock_unidades.to_excel(writer, sheet_name='S_StockUnidades', index=False)


        # === HOJA RESUMEN CON KPIs ===
        kpis = indicadores(model)  # Usar tu función de KPIs

        # Crear DataFrame con resumen de KPIs
        kpi_data = [
            ['Costo total', kpis.get('FO', 0)],
            ['Costo Transporte', kpis.get('costo_transporte', 0)],
            ['Costo Almacenamiento', kpis.get('costo_almacenamiento', 0)],
            ['Total Viajes', kpis.get('total_viajes', 0)],
            ['Exceso Capacidad Peso', kpis.get('pesoexc', 0)],
            ['Exceso Capacidad Volumen', kpis.get('volexc', 0)],
            ['% Viajes Tercerizados', kpis.get('viajesterc', 0)],
            ['% Viajes Propios', kpis.get('viajesprop', 0)],
            ['% Gasto Tercerización', kpis.get('gastospot', 0)],
            ['Km en vacío', kpis.get('kmvacio', 0)],
            ['% Km en Vacío', kpis.get('propkmvacio', 0)]
        ]

        df_kpis = pd.DataFrame(kpi_data, columns=['Indicador', 'Valor'])
        df_kpis.to_excel(writer, sheet_name='RESUMEN_KPIs', index=False)

        # Uso por tipo de vehículo (si existe)
        if 'usotipo' in kpis and kpis['usotipo']:
            usotipo_data = []
            for vehicle, usage in kpis['usotipo'].items():
                usotipo_data.append([vehicle, usage])

            df_usotipo = pd.DataFrame(usotipo_data, columns=['Tipo_Vehículo', 'Uso'])
            if usotipo_data:
                df_usotipo.to_excel(writer, sheet_name='RESUMEN_KPIs', startrow=len(df_kpis)+3, index=False)

        # Utilización por arco (si existe)
        if 'utiliz_peso' in kpis and kpis['utiliz_peso']:
            utiliz_data = []
            for arc, util in kpis['utiliz_peso'].items():
                utiliz_data.append([f"{arc[0]}-{arc[1]}", util])

            df_utiliz = pd.DataFrame(utiliz_data, columns=['Arco', 'Utilización_Peso'])
            if 'usotipo' in kpis and kpis['usotipo']:
                df_utiliz.to_excel(writer, sheet_name='RESUMEN_KPIs', startrow=len(df_kpis)+len(df_usotipo)+6, index=False)
            else:
                df_utiliz.to_excel(writer, sheet_name='RESUMEN_KPIs', startrow=len(df_kpis)+6, index=False)

    print(f"Archivo de resultados exportado a: {archivo}")
    return archivo



---



---
# Cargue de los datos
En esta sección se hará la lectura de los datos (que están en formato xlsx), y su debida conversión a diccionarios. Esto debido a su facilidad para tratarlos y posteriormente definir los sets y parámetros que alimentarán el modelo.

In [ ]:
# Define el nombre de archivo deseado (como respaldo/referencia)
nombre_archivo_deseado = '20250918-estructura de datos-V4 (2) .xlsx'

# Asegura que nombre_archivo_subido existe en el ámbito global
if 'nombre_archivo_subido' not in globals():
    nombre_archivo_subido = None


if os.path.exists(nombre_archivo_deseado):
    # Verifica si el archivo deseado existe en el directorio actual
    print(f"Archivo '{nombre_archivo_deseado}' encontrado localmente. Cargando datos...")
    archivo_excel = pd.ExcelFile(nombre_archivo_deseado)
    nombre_archivo_subido = nombre_archivo_deseado # Actualiza nombre_archivo_subido para futuras ejecuciones
elif nombre_archivo_subido and os.path.exists(nombre_archivo_subido):
    print(f"Archivo '{nombre_archivo_subido}' de una ejecución anterior encontrado localmente. Cargando datos...")
    archivo_excel = pd.ExcelFile(nombre_archivo_subido)
else:
    print(f"Archivo '{nombre_archivo_deseado}' no encontrado localmente. Por favor, sube el archivo.")
    uploaded = files.upload()

    if uploaded:
        # Obtiene el nombre del primer (y probablemente único) archivo subido
        nombre_archivo_subido = list(uploaded.keys())[0]
        print(f"Archivo '{nombre_archivo_subido}' subido correctamente. Cargando datos...")
        archivo_excel = pd.ExcelFile(io.BytesIO(uploaded[nombre_archivo_subido]))
    else:
        print("No se subió ningún archivo.")
        archivo_excel = None

if nombre_archivo_subido and archivo_excel:
  print(f"Nombre del archivo de datos utilizado: {nombre_archivo_subido}")

Archivo '20250918-estructura de datos-V4 (2) .xlsx' no encontrado localmente. Por favor, sube el archivo.


Saving 20250918-estructura de datos-V4 (2) ajustada.xlsx to 20250918-estructura de datos-V4 (2) ajustada.xlsx
Archivo '20250918-estructura de datos-V4 (2) ajustada.xlsx' subido correctamente. Cargando datos...
Nombre del archivo de datos utilizado: 20250918-estructura de datos-V4 (2) ajustada.xlsx


In [ ]:
archivo_excel.sheet_names

['Conjuntos',
 'Nodos',
 'Materiales',
 'MaterialesNodos',
 'MaterialesTransito',
 'Vehiculos',
 'VehiculosNodos',
 'VehiculosTransito',
 'Arcos',
 'Demanda',
 'CapacidadPdn']

In [ ]:
verificar_hojas_excel(archivo_excel)

Verificación de hojas:
100% de hojas requeridas encontradas: ■■■■■■■■■■
✅ Todas las hojas requeridas están presentes.


In [ ]:
data = leer_datos(archivo_excel)


--------------------------------------------------------------------------------
Lectura de la hoja Conjuntos

Nodos: 24
Materiales: 2
Condiciones de transporte: 4
Tipos de vehiculos: 16
Tipos de remolques: 3
Tipos de cabezotes: 1
Tiempo: 10
Arcos: 45

--------------------------------------------------------------------------------
Lectura de la hoja Nodos

Dimensión del dataframe extraído: 24 filas y 4 columnas
✅ Verificación de consistencia exitosa
24 llaves únicas: ['Buga_Molino_Santa_Marta', 'Santa_Marta_Molino_Santa_Marta', 'Planta_Noel_Medellin', 'Aguachica', 'Barranquilla', 'Bogotá', 'Bucaramanga', 'Cali', 'Carmen_de_Viboral', 'Cartagena', 'Cucuta', 'Duitama', 'Florencia', 'Ibague', 'Medellin', 'Monteria', 'Neiva', 'Pasto', 'Pereira', 'Sincelejo', 'Uramita', 'Valledupar', 'Villavicencio', 'Yopal']



,N,qr_n,qd_n,capparq_n
0,Buga_Molino_Santa_Marta,9999,7,9999
1,Santa_Marta_Molino_Santa_Marta,9999,7,9999
2,Planta_Noel_Medellin,10,48,14
3,Aguachica,9999,9999,9999
4,Barranquilla,9999,9999,9999



--------------------------------------------------------------------------------
Lectura de la hoja Materiales

Dimensión del dataframe extraído: 2 filas y 9 columnas
✅ Verificación de consistencia exitosa
2 llaves únicas: ['Galletas_Seco', 'Harina_MP_Seco']



,M,condmat_c1m,condmat_c2m,condmat_c3m,condmat_c4m,unid_m,peso_m,vol_m,costoinv_m
0,Galletas_Seco,1,0,0,0,unidades,0.37543,0.00176,2785
1,Harina_MP_Seco,1,0,0,0,big bag,900.00000,1.00000,5000000



--------------------------------------------------------------------------------
Lectura de la hoja MaterialesNodos

Dimensión del dataframe extraído: 22 filas y 5 columnas
✅ Verificación de consistencia exitosa
22 llaves únicas: {('Galletas_Seco', 'Pasto'), ('Galletas_Seco', 'Pereira'), ('Galletas_Seco', 'Villavicencio'), ('Galletas_Seco', 'Yopal'), ('Galletas_Seco', 'Aguachica'), ('Galletas_Seco', 'Valledupar'), ('Galletas_Seco', 'Cartagena'), ('Galletas_Seco', 'Bogotá'), ('Galletas_Seco', 'Cali'), ('Galletas_Seco', 'Cucuta'), ('Galletas_Seco', 'Uramita'), ('Galletas_Seco', 'Barranquilla'), ('Galletas_Seco', 'Medellin'), ('Galletas_Seco', 'Monteria'), ('Galletas_Seco', 'Ibague'), ('Galletas_Seco', 'Bucaramanga'), ('Galletas_Seco', 'Neiva'), ('Galletas_Seco', 'Florencia'), ('Harina_MP_Seco', 'Planta_Noel_Medellin'), ('Galletas_Seco', 'Sincelejo'), ('Galletas_Seco', 'Duitama'), ('Galletas_Seco', 'Carmen_de_Viboral')}
    formadas por combinaciones entre los conjuntos Materiales y Nodo

...

""



--------------------------------------------------------------------------------
Lectura de la hoja VehiculosTransito

Dimensión del dataframe extraído: 0 filas y 0 columnas
DataFrame para VehiculosTransito está vacío. Agregando entrada de placeholder.
('C2S2_FURGON_T', 'Buga_Molino_Santa_Marta', 1)
Placeholder key added: ('C2S2_FURGON_T', 'Buga_Molino_Santa_Marta', 1)
✅ Verificación de consistencia exitosa
1 llaves únicas: {('C2S2_FURGON_T', 'Buga_Molino_Santa_Marta', 1)}
    formadas por combinaciones entre los conjuntos Vehiculos y Nodos
    Vehiculos: ['C2S2_FURGON_T']
    Nodos: ['Buga_Molino_Santa_Marta']
    Tiempo: [1]



""



--------------------------------------------------------------------------------
Lectura de la hoja CapacidadPdn

Dimensión del dataframe extraído: 3 filas y 3 columnas
✅ Verificación de consistencia exitosa
3 llaves únicas: {('Harina_MP_Seco', 'Buga_Molino_Santa_Marta'), ('Harina_MP_Seco', 'Santa_Marta_Molino_Santa_Marta'), ('Galletas_Seco', 'Planta_Noel_Medellin')}
    formadas por combinaciones entre los conjuntos CapacidadPdn y Nodos
    CapacidadPdn: ['Harina_MP_Seco', 'Galletas_Seco']
    Nodos: ['Santa_Marta_Molino_Santa_Marta', 'Buga_Molino_Santa_Marta', 'Planta_Noel_Medellin']



,M,N,qp_mn
0,Galletas_Seco,Planta_Noel_Medellin,10500400
1,Harina_MP_Seco,Buga_Molino_Santa_Marta,140000
2,Harina_MP_Seco,Santa_Marta_Molino_Santa_Marta,140000


In [ ]:
display(data)

{'Conjuntos': {'Nodos': {'tamaño': 24},
  'Materiales': {'tamaño': 2},
  'Condiciones de transporte': {'tamaño': 4},
  'Tipos de vehiculos': {'tamaño': 16},
  'Tipos de remolques': {'tamaño': 3},
  'Tipos de cabezotes': {'tamaño': 1},
  'Tiempo': {'tamaño': 10},
  'Arcos': {'tamaño': 45}},
 'Nodos': {'Buga_Molino_Santa_Marta': {'qr_n': 9999,
   'qd_n': 7,
   'capparq_n': 9999},
  'Santa_Marta_Molino_Santa_Marta': {'qr_n': 9999,
   'qd_n': 7,
   'capparq_n': 9999},
  'Planta_Noel_Medellin': {'qr_n': 10, 'qd_n': 48, 'capparq_n': 14},
  'Aguachica': {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999},
  'Barranquilla': {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999},
  'Bogotá': {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999},
  'Bucaramanga': {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999},
  'Cali': {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999},
  'Carmen_de_Viboral': {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999},
  'Cartagena': {'qr_n': 9999, 'qd_n': 9999, 'capparq_n': 9999},
  'Cucuta

---
# Implementación

In [ ]:
list(data.keys())

['Conjuntos',
 'Nodos',
 'Materiales',
 'MaterialesNodos',
 'Vehiculos',
 'U',
 'K',
 'R',
 'VehiculosNodos',
 'Arcos',
 'Demanda',
 'T',
 'MaterialesTransito',
 'VehiculosTransito',
 'CapacidadPdn']

In [ ]:
# ---------------------------
# Conjuntos
# ---------------------------


N = list(data['Nodos'].keys())                          # Conjunto de nodos (ubicaciones geográficas)
N_p = [key[1] for key in data['CapacidadPdn'].keys()]   # Nodos productores
M = list(data['Materiales'].keys())                     # Conjunto de materiales/productos
M_n = {}
for n in N:
    M_n[n] = []
for material, nodo in data['MaterialesNodos'].keys():
    if nodo in M_n:
        M_n[nodo].append(material)
C = ['seco', 'refrigerado', 'congelado', 'ultracongelado']  # Conjunto de condiciones de transporte
V = list(data['Vehiculos'].keys())                      # Conjunto de vehículos expandidos (formato: id-condicion)
V_p = [v for v, attrs in data['Vehiculos'].items() if attrs.get('ownership') in ['propio', 'Propio']]  # Vehículos propios
R = list(data['R'])                                     # Conjunto de remolques
K = list(data['K'])                                     # Conjunto de cabezotes/tractocamiones
U = list(data['U'])                                     # Conjunto de unidades de transporte (K ∪ R)
T = [t for t in data['T']]                         # Conjunto de períodos de tiempo
A = list((data['Arcos'][a]['i'], data['Arcos'][a]['j']) for a in data['Arcos'].keys())  # Conjunto de arcos (origen, destino)



# ---------------------------
# Parametros
# ---------------------------


# ------- Materiales -------

# Volumen unitario por material (m³/unidad)
vol_m = {m: data['Materiales'][m]['vol_m'] for m in data['Materiales'].keys()}

# Peso unitario por material (kg/unidad)
peso_m = {m: data['Materiales'][m]['peso_m'] for m in data['Materiales'].keys()}

# Compatibilidad material-condición (1 si material m puede transportarse en condición c, 0 en caso contrario)
condmat_cm = {c: {m: data['Materiales'][m].get(f'condmat_c{C.index(c)+1}m', 0) for m in M} for c in C}


# ------- Inventario -------

# Inventario inicial de material m en nodo n
inv_mn = {m: {n: data['MaterialesNodos'].get((m, n), {}).get('inv_mn', 0) for n in N} for m in M}
# inv_mn = {m: {n: 2*data['Demanda'].get((m, n, T[0]), 0) for n in N} for m in M}

# Inventario en tránsito de material m desde nodo n en período t
invt_mnt = {m: {n: {t: data['MaterialesTransito'].get((m, n, t), 0) for t in T} for n in N} for m in M}

# Capacidad máxima de inventario de material m en nodo n
invmax_mn = {m: {n: data['MaterialesNodos'].get((m, n), {}).get('invmax_mn', 0) for n in N} for m in M}

# Inventario mínimo requerido de material m en nodo n
invmin_mn = {m: {n: data['MaterialesNodos'].get((m, n), {}).get('invmin_mn', 0) for n in N} for m in M}

# Costo de mantener inventario por unidad de material m
costoinv_m = {m: data['Materiales'][m]['costoinv_m'] for m in data['Materiales'].keys()}


# ------- Unidades de transporte -------

# Disponibilidad inicial de unidad u en nodo n
vehi_un = {u: {n: data['VehiculosNodos'].get((u, n), {}).get('vehi_un', 0) for n in N} for u in U}

# Disponibilidad en tránsito de unidad u desde nodo n en período t
veht_unt = {u: {n: {t: sum(data['VehiculosTransito'].get((v, n, t), 0) * (1 if (data['Vehiculos'][v].get('K') == u or data['Vehiculos'][v].get('R') == u) else 0) for v in data['Vehiculos'].keys()) for t in T} for n in N} for u in U}


# ------- Capacidades de nodos -------

# Capacidad de recepción del nodo n (unidades/período)
qr_n = {n: data['Nodos'][n]['qr_n'] for n in data['Nodos'].keys()}

# Capacidad de despacho del nodo n (unidades/período)
qd_n = {n: data['Nodos'][n]['qd_n'] for n in data['Nodos'].keys()}

# Capacidad máxima de parqueo en nodo n (por defecto ilimitada)
capparq_n = {n: 9999 for n in data['Nodos'].keys()}

# Capacidad de producción de nodos productores
qp_mn = {m: {n: data['CapacidadPdn'].get((m, n), {}).get('qp_mn', 0) for n in N} for m in M}


# ------- Capacidades de vehículos -------

# Capacidad volumétrica del vehículo v (m³)
qvol_v = {v: data['Vehiculos'][v]['qvol_v'] for v in data['Vehiculos'].keys()}

# Capacidad de peso del vehículo v (kg)
qpes_v = {v: data['Vehiculos'][v]['qpes_v'] for v in data['Vehiculos'].keys()}


# ------- Compatibilidad y composición -------

# Compatibilidad vehículo-condición (1 si vehículo v puede manejar condición c, 0 en caso contrario)
condveh_cv = {c: {v: 1 if (data['Vehiculos'][v].get('cond') == c) else 0 for v in data['Vehiculos']} for c in C}

# Composición vehículo-unidad para vehículos propios (1 si vehículo v usa unidad u, 0 en caso contrario)
composic_vu = {v: {u: 1 if (data['Vehiculos'][v].get('K') == u or data['Vehiculos'][v].get('R') == u) else 0 for u in U} for v in V_p}


# ------- Demanda -------

# Demanda de material m en nodo n durante período t
dem_mnt = {m: {n: {t: data['Demanda'].get((m, n, t), 0) for t in T} for n in N} for m in M}


# ------- Arcos -------

# Costo de viaje con carga en el arco (i, j) con el tipo de vehículo v
temp = {(data['Arcos'][a]['i'], data['Arcos'][a]['j'], v): data['Arcos'][a]['vehicles'][v][list(data['Arcos'][a]['vehicles'][v].keys())[0]]['costo_ijvc'] for a in data['Arcos'].keys() for v in data['Arcos'][a]['vehicles'].keys()}
costotrans_ijv = {i: {j: {v: temp.get((i, j, v), 10**10) for v in V} for j in N if (i, j) in A} for i in N}

# Costo de viaje vacío en el arco (i, j) con el tipo de vehículo v
temp = {(data['Arcos'][a]['i'], data['Arcos'][a]['j'], v): data['Arcos'][a]['vehicles'][v][list(data['Arcos'][a]['vehicles'][v].keys())[0]]['costov_ijvc'] for a in data['Arcos'].keys() for v in data['Arcos'][a]['vehicles'].keys()}
costotransvac_ijv = {i: {j: {v: temp.get((i, j, v), 99999999) for v in V} for j in N if (i, j) in A} for i in N} #Default de 9999?

# Tiempo de viaje en el arco (i, j)
temp = {(data['Arcos'][a]['i'], data['Arcos'][a]['j']): data['Arcos'][a]['vehicles'][list(data['Arcos'][a]['vehicles'].keys())[0]][list(data['Arcos'][a]['vehicles'][list(data['Arcos'][a]['vehicles'].keys())[0]].keys())[0]]['tiempo_ijvc'] for a in data['Arcos'].keys()}
lt_ij = {i: {j: temp.get((i, j), 99) for j in N if (i, j) in A} for i in N} #Default de 99?

# Capacidad del arco (i, j) del tipo de vehículo v
temp = {(data['Arcos'][a]['i'], data['Arcos'][a]['j'], v): data['Arcos'][a]['vehicles'][v][list(data['Arcos'][a]['vehicles'][v].keys())[0]]['caparco_ijv'] for a in data['Arcos'].keys() for v in data['Arcos'][a]['vehicles'].keys()}
caparco_ijv = {i: {j: {v: temp.get((i, j, v), 9999) for v in V} for j in N if (i, j) in A} for i in N} #Default de 9999? Ponemos todas las combinaciones o soo las existentes en A?  -- # if f'a{i[1:]}{j[1:]}' in A

# Distancia para KPIs
temp = {(data['Arcos'][a]['i'], data['Arcos'][a]['j']): data['Arcos'][a]['vehicles'][list(data['Arcos'][a]['vehicles'].keys())[0]][list(data['Arcos'][a]['vehicles'][list(data['Arcos'][a]['vehicles'].keys())[0]].keys())[0]]['dist_ijv'] for a in data['Arcos'].keys()}
dist_ij = {i: {j: temp.get((i, j), 9999) for j in N if (i, j) in A} for i in N} #Default de 99?


In [ ]:
es_factible, errores, advertencias = ejecutar_verificaciones_completas()


=== RESUMEN DE DATOS DEL PROBLEMA ===
Nodos: 24 (Productores: 3)
Materiales: 2
Vehículos: 16 (Propios: 4, Tercerizados: 12)
Condiciones: 4
Cabezotes: 2, Remolques: 6
Períodos: 10
Arcos: 45

Nodos productores: ['Planta_Noel_Medellin', 'Buga_Molino_Santa_Marta', 'Santa_Marta_Molino_Santa_Marta']
Condiciones: ['seco', 'refrigerado', 'congelado', 'ultracongelado']

Demanda total del material Galletas_Seco en el horizonte: 6379133.00
Capacidad productiva total del material Galletas_Seco: 105004000.00

Demanda total del material Harina_MP_Seco en el horizonte: 3020.00
Capacidad productiva total del material Harina_MP_Seco: 2800000.00

==== VERIFICACIONES DE FACTIBILIDAD ===

1. Verificando inventario inicial, en tránsito y demanda...
2. Verificando coherencia de límites de inventario...
3. Verificando capacidades de vehículos...
4. Verificando conectividad de la red...
5. Verificando balance oferta-demanda...
6. Verificando compatibilidad material-condición-vehículo...
   ✓ Material Galleta

In [ ]:
# ---------------------------
# Modelo
# ---------------------------


model = pl.LpProblem("Modelo_v1", pl.LpMinimize)


# ---------------------------
# Variables de decisión
# ---------------------------


# Cantidad de material m a enviar por el arco (i, j) en el periodo t
X = pl.LpVariable.dicts("X", ((i, j, m, t) for (i, j) in A for m in M for t in T),
                        lowBound=0, cat=pl.LpContinuous)

# Cantidad de vehículos propios v transitando el arco (i, j) en el periodo t
YP = pl.LpVariable.dicts("YP", ((i, j, v, t) for (i, j) in A for v in V_p for t in T),
                        lowBound=0, cat=pl.LpInteger)

# Cantidad de vehículos tercerizados v transitando el arco (i, j) en el periodo t
YT = pl.LpVariable.dicts("YT", ((i, j, v, t) for (i, j) in A for v in V if v not in V_p for t in T),
                        lowBound=0, cat=pl.LpInteger)

# Cantidad de vehículos vacíos transitadno el aro (i, j) usando el vehículo propio v en el periodo t
W = pl.LpVariable.dicts("W", ((i, j, v, t) for (i, j) in A for v in V_p for t in T),
                        lowBound=0, cat=pl.LpInteger)

# Cantidad de elementos de U transitando el arco (i, j) en el periodo t
B = pl.LpVariable.dicts("B", ((i, j, u, t) for (i, j) in A for u in U for t in T),
                        lowBound=0, cat=pl.LpInteger)

# Nivel de material m  en el nodo n en el periodo t
I = pl.LpVariable.dicts("I", ((m, n, t) for m in M for n in N for t in T),
                        lowBound=0, cat=pl.LpContinuous)

# Cantidad de unidades de transporte del tipo u en el nodo n en el periodo t
S = pl.LpVariable.dicts("S", ((u, n, t) for u in U for n in N for t in T),
                        lowBound=0, cat=pl.LpInteger)


# ---------------------------
# Restricciones
# ---------------------------

# for j in ['Buga - Molino Santa Marta', 'Santa Marta - Molino Santa Marta']:
#     model += pl.lpSum(X[i, j, m, t] for i in N if (i, j) in A for m in M for t in T) == 0

# model += pl.lpSum(X[i,'Planta Noel Medellin','Galletas_Seco', t] for i in N if (i, 'Planta Noel Medellin') in A for t in T) == 0

# Inventario inicial
for m in M:
    for n in N:
        if m in M_n[n]:
            model += inv_mn[m][n] + invt_mnt[m][n][T[0]] - dem_mnt[m][n][T[0]] == I[m, n, T[0]] + pl.lpSum(X[n, j, m, T[0]] for j in N if (n, j) in A)

# Balance de inventario
for m in M:
    for n in N:
        for t_idx, t in enumerate(T[1:], 1):
            if m in M_n[n]:
              model += I[m, n, T[t_idx - 1]] + invt_mnt[m][n][t] + pl.lpSum(X[i, n, m, T[t_idx-lt_ij[i][n]]] for i in N if (i, n) in A)- dem_mnt[m][n][t] == I[m, n, t] + pl.lpSum(X[n, j, m, t] for j in N  if (n, j) in A)

# Capacidad de producción nodos productores de lo que producen
for m in M:
    for n in N:#_p:
        for t in T:
            # if m not in M_n[n]:
            model += pl.lpSum(X[n, j, m, t] for j in N if (n, j) in A) <= qp_mn[m][n]

# Capacidad de envío de nodos productores de lo que no producen
# for m in M:
#     for n in N_p:
#         for t in T:
#           if m in M_n[n]:
#               model += pl.lpSum(X[n, j, m, t] for j in N if (n, j) in A) <= 0


# Rango del inventario
for m in M:
    for n in N:
        for t in T:
          if m in M_n[n]:
              model += I[m, n, t] <= invmax_mn[m][n]
              model += I[m, n, t] >= invmin_mn[m][n]

# Inventario inicial de U (cabezotes y remolques)
for u in U:
    for n in N:
        model += vehi_un[u][n] + veht_unt[u][n][T[0]] == S[u, n, T[0]] +  pl.lpSum(B[n, j, u, T[0]] for j in N if (n, j) in A)

# Balance de masa del inventario de U (cabezotes y remolques)
for u in U:
    for n in N:
        for t_idx, t in enumerate(T[1:], 1):
            model += S[u, n, T[t_idx - 1]] + veht_unt[u][n][t] + pl.lpSum(B[i, n, u, T[t_idx-lt_ij[i][n]]] for i in N  if (i, n) in A) == S[u, n, t] + pl.lpSum(B[n, j, u, t] for j in N  if (n, j) in A)

# Configuraciones de los vehículos
for (i, j) in A:
    for u in U:
        for t in T:
            model += pl.lpSum(composic_vu[v][u]*(YP[i, j, v, t]+W[i, j, v, t]) for v in V_p) == B[i, j, u, t]

# Capacidad en peso
for (i, j) in A:
    for c in C:
        for t in T:
            model += pl.lpSum(condmat_cm[c][m]*peso_m[m]*X[i, j, m, t] for m in M) <= pl.lpSum(qpes_v[v]*condveh_cv[c][v]*YP[i, j, v, t] for v in V_p) + pl.lpSum(qpes_v[v]*condveh_cv[c][v]*YT[i, j, v, t] for v in V if v not in V_p)

# Capacidad en volumen
for (i, j) in A:
    for c in C:
        for t in T:
            model += pl.lpSum(condmat_cm[c][m]*vol_m[m]*X[i, j, m, t] for m in M) <= pl.lpSum(qvol_v[v]*condveh_cv[c][v]*YP[i, j, v, t] for v in V_p) + pl.lpSum(qvol_v[v]*condveh_cv[c][v]*YT[i, j, v, t] for v in V if v not in V_p)

# Capacidad de los arcos
for (i, j) in A:
    for v in V_p:
        for t in T:
            model += YP[i, j, v, t] + W[i, j, v, t] <= caparco_ijv[i][j][v]

for (i, j) in A:
    for v in V:
        if v not in V_p:
            for t in T:
                model += YT[i, j, v, t] <= caparco_ijv[i][j][v]

# Capacidad de recepción
for n in N:
    for t in T:
        model += pl.lpSum(YP[i, n, v, t] for i in N if (i, n) in A for v in V_p) + pl.lpSum(YT[i, n, v, t] for i in N if (i, n) in A for v in V if v not in V_p) <= qr_n[n]

# Capacidad de despacho
for n in N:
    for t in T:
        model += pl.lpSum(YP[n, j, v, t] for j in N if (n, j) in A for v in V_p) + pl.lpSum(YT[n, j, v, t] for j in N if (n, j) in A for v in V if v not in V_p) <= qd_n[n]


# ---------------------------
# Función objetivo
# ---------------------------

transprop = pl.lpSum(costotrans_ijv[i][j][v]*YP[i, j, v, t] + costotransvac_ijv[i][j][v]*W[i, j, v, t] for i in N for j in N if (i, j) in A for t in T for v in V_p)
transterc = pl.lpSum(costotrans_ijv[i][j][v]*YT[i, j, v, t] for i in N for j in N if (i, j) in A for t in T for v in V if v not in V_p)
almacenamiento = pl.lpSum(costoinv_m[m]*I[m, n, t] for m in M for n in N for t in T)

model += transprop + transterc + almacenamiento


# ---------------------------
# Configuración del solver
# ---------------------------
# cbc = pl.PULP_CBC_CMD(msg=1, timeLimit=30)  # concise logs, 30s cap, 1% gap
# model.solve(cbc)

options = {
    "WLSACCESSID": "5fbe455a-d58f-4dd3-a143-3733a1a1df56",
    "WLSSECRET": "8ce24b99-1b93-4b60-b4f6-0ce853372535",
    "LICENSEID": 2677405,
    "TimeLimit": 600
}
with gp.Env(params=options) as env:
    solver = pl.GUROBI(env=env)
    model.solve(solver)
    solver.close()

# ---------------------------
# Salidas
# ---------------------------
print("Status:", pl.LpStatus[model.status])
print("Objective:", pl.value(model.objective))

Set parameter TimeLimit to value 600
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2677405
Academic license 2677405 - for non-commercial use only - registered to da___@udea.edu.co
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  600

Academic license 2677405 - for non-commercial use only - registered to da___@udea.edu.co
Optimize a model with 17940 rows, 15900 columns and 63445 nonzeros
Model fingerprint: 0x65831cb5
Variable types: 1380 continuous, 14520 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-03, 3e+04]
  Objective range  [3e+03, 1e+10]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issu

In [ ]:
# for v in model.variables():
#     if v.varValue is not None and v.varValue > 0:
#         print(f"{v.name}: {v.varValue}")

## Indicadores

In [ ]:
indicadores(model)

{'FO': 113620395188.16,
 'costo_transporte': 919713443.1599997,
 'costo_almacenamiento': 112700681745.0,
 'total_viajes': 458.0,
 'utiliz_peso': {('Planta_Noel_Medellin', 'Aguachica'): 0.1447394211886305,
  ('Planta_Noel_Medellin', 'Barranquilla'): 0.6464380744186047,
  ('Planta_Noel_Medellin', 'Bogotá'): 0.7206251803346007,
  ('Planta_Noel_Medellin', 'Bucaramanga'): 0.44402794873385004,
  ('Planta_Noel_Medellin', 'Cali'): 0.344352145088968,
  ('Planta_Noel_Medellin', 'Carmen_de_Viboral'): 0.6010217155555556,
  ('Planta_Noel_Medellin', 'Cartagena'): 0.34214575374677003,
  ('Planta_Noel_Medellin', 'Cucuta'): 0.2348406809819122,
  ('Planta_Noel_Medellin', 'Duitama'): 0.24195541901808784,
  ('Planta_Noel_Medellin', 'Florencia'): 0.17562751214470282,
  ('Planta_Noel_Medellin', 'Ibague'): 0.2980817189664082,
  ('Planta_Noel_Medellin', 'Monteria'): 0.33802087426356586,
  ('Planta_Noel_Medellin', 'Neiva'): 0.2764522944702842,
  ('Planta_Noel_Medellin', 'Pasto'): 0.31245088992248055,
  ('Plant

## Exportar resultados a archivo de excel

In [ ]:
exportar_resultados_excel(model, archivo='resultados.xlsx')

Archivo de resultados exportado a: resultados.xlsx


'resultados.xlsx'

## Análisis de restricciones constrictivas

In [ ]:
# model.writeLP("modelo.lp")

# options = {
#     "WLSACCESSID": "5fbe455a-d58f-4dd3-a143-3733a1a1df56",
#     "WLSSECRET": "8ce24b99-1b93-4b60-b4f6-0ce853372535",
#     "LICENSEID": 2677405,
#     "TimeLimit": 600
# }

# with gp.Env(params=options) as env:
#     # Crear modelo en este entorno
#     modelo = gp.read("modelo.lp", env=env)

#     # Resolver
#     modelo.optimize()

#     if modelo.status == GRB.INFEASIBLE:
#         print("\nModelo infactible. Generando IIS...\n")
#         modelo.computeIIS()
#         modelo.write("modelo.ilp")

#         print("Archivo IIS generado: modelo.ilp\n")
#         print("Restricciones y variables conflictivas:")

#         for c in modelo.getConstrs():
#             if c.IISConstr:
#                 print(f" - Restricción en IIS: {c.constrName}")

#         for v in modelo.getVars():
#             if v.IISLB:
#                 print(f" - Variable {v.varName} tiene LB conflictivo")
#             if v.IISUB:
#                 print(f" - Variable {v.varName} tiene UB conflictivo")
#         with open("modelo.ilp", 'r') as f:
#             for i in range(150): # Read and print the first 150 lines
#                 line = f.readline()
#                 if not line:
#                     break
#                 print(line.strip())

#     else:
#         print("El modelo no es infactible. Status:", modelo.status)



## Otros solvers

In [ ]:
# GLPK
# !apt-get install -y -qq glpk-utils

# CUOPT
# !pip install --extra-index-url=https://pypi.nvidia.com \
#     cuopt-cu12==25.5.* \
#     cuopt-sh-client==25.5.* \
#     nvidia-cuda-runtime-cu12==12.8.*

# SCIP
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install -y -c conda-forge pyscipopt scip




---

